# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 72 new papers today
          33 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/32 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.09082


extracting tarball to tmp_2302.09082...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.09091


extracting tarball to tmp_2302.09091... done.
Retrieving document from  https://arxiv.org/e-print/2302.09199


...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.10048


extracting tarball to tmp_2302.10048...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.10107


extracting tarball to tmp_2302.10107...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10008-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10008) | **TOI-3235 b: a transiting giant planet around an M4 dwarf star**  |
|| <mark>Melissa J. Hobson</mark>, et al. -- incl., <mark>Th. Henning</mark>, <mark>Richard P. Schwarz</mark> |
|*Appeared on*| *2023-02-21*|
|*Comments*| *15 pages, 4 figures. Accepted for publication in APJL*|
|**Abstract**| We present the discovery of TOI-3235 b, a short-period Jupiter orbiting an M-dwarf with a stellar mass close to the critical mass at which stars transition from partially to fully convective. TOI-3235 b was first identified as a candidate from TESS photometry, and confirmed with radial velocities from ESPRESSO, and ground-based photometry from HATSouth, MEarth-South, TRAPPIST-South, LCOGT, and ExTrA. We find that the planet has a mass of $\mathrm{0.665\pm0.025\,M_J}$ and a radius of $\mathrm{1.017\pm0.044\,R_J}$. It orbits close to its host star, with an orbital period of $\mathrm{2.5926\,d}$, but has an equilibrium temperature of $\mathrm{\approx 604 \, K}$, well below the expected threshold for radius inflation of hot Jupiters. The host star has a mass of $\mathrm{0.3939\pm0.0030\,M_\odot}$, a radius of $\mathrm{0.3697\pm0.0018\,R_\odot}$, an effective temperature of $\mathrm{3389 \, K}$, and a J-band magnitude of $\mathrm{11.706\pm0.025}$. Current planet formation models do not predict the existence of gas giants such as TOI-3235 b around such low-mass stars. With a high transmission spectroscopy metric, TOI-3235 b is one of the best-suited giants orbiting M-dwarfs for atmospheric characterization. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.09199-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.09199) | **Abundance matching analysis of the emission line galaxy sample in the  extended Baryon Oscillation Spectroscopic Survey**  |
|| <mark>Sicheng Lin</mark>, et al. -- incl., <mark>Hong Guo</mark>, <mark>Joel R. Brownstein</mark> |
|*Appeared on*| *2023-02-21*|
|*Comments*| *MNRAS, Volume 519, Issue 3*|
|**Abstract**| We present the measurements of the small-scale clustering for the emission line galaxy (ELG) sample from the extended Baryon Oscillation Spectroscopic Survey (eBOSS) in the Sloan Digital Sky Survey IV (SDSS-IV). We use conditional abundance matching method to interpret the clustering measurements from $0.34h^{-1}\textrm{Mpc}$ to $70h^{-1}\textrm{Mpc}$. In order to account for the correlation between properties of emission line galaxies and their environment, we add a secondary connection between star formation rate of ELGs and halo accretion rate. Three parameters are introduced to model the ELG [OII] luminosity and to mimic the target selection of eBOSS ELGs. The parameters in our models are optimized using Markov Chain Monte Carlo (MCMC) method. We find that by conditionally matching star formation rate of galaxies and the halo accretion rate, we are able to reproduce the eBOSS ELG small scale clustering within 1$\sigma$ error level. Our best fit model shows that the eBOSS ELG sample only consists of $\sim 12\%$ of all star-forming galaxies, and the satellite fraction of eBOSS ELG sample is 19.3\%. We show that the effect of assembly bias is $\sim20\%$ on the two-point correlation function and $\sim5\%$ on the void probability function at scale of $r\sim 20 h^{-1}\rm Mpc$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.09091-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.09091) | **Neutrino-Assisted Early Dark Energy is a Natural Resolution of the  Hubble Tension**  |
|| Mariana Carrillo González, <mark>Qiuyue Liang</mark>, Jeremy Sakstein, Mark Trodden |
|*Appeared on*| *2023-02-21*|
|*Comments*| *5 pages, 1 figure, comment*|
|**Abstract**| It has very recently been claimed that the neutrino-assisted early dark energy model -- a promising resolution of the Hubble tension that can ameliorate the theoretical fine-tuning and coincidence problems that plague other theories -- does not provide natural or cosmologically interesting results. In this short paper, we show that these conclusions are incorrect for three reasons. First, we identify errors in the calculations. Second, we dispute the definition in of what constitutes an 'interesting' and 'natural' model. Finally, we demonstrate that the conclusions of were arrived at without fully exploring the full parameter space of the model. Neutrino-assisted early dark energy remains a natural and interesting potential resolution of the Hubble tension that merits further study. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10048-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10048) | **GRBAlpha: the smallest astrophysical space observatory -- Part 1:  Detector design, system description and satellite operations**  |
|| András Pál, et al. -- incl., <mark>Pavol Lipovský</mark> |
|*Appeared on*| *2023-02-21*|
|*Comments*| *Submitted to Astronomy & Astrophysics, 8 pages, 9 figures*|
|**Abstract**| Aims. Since launched on 2021 March 22, the 1U-sized CubeSat GRBAlpha operates and collects scientific data on high-energy transients, making it the smallest astrophysical space observatory to date. GRBAlpha is an in-obit demonstration of a gamma-ray burst (GRB) detector concept suitably small to fit into a standard 1U volume. As it was demonstrated in a companion paper, GRBAlpha adds significant value to the scientific community with accurate characterization of bright GRBs, including the recent event of GRB 221009A. Methods. The GRB detector is a 75x75x5 mm CsI(Tl) scintillator wrapped in a reflective foil (ESR) read out by an array of SiPM detectors, multi-pixel photon counters by Hamamatsu, driven by two separate, redundant units. To further protect the scintillator block from sunlight and protect the SiPM detectors from particle radiation, we apply a multi-layer structure of Tedlar wrapping, anodized aluminium casing and a lead-alloy shielding on one edge of the assembly. The setup allows observations of gamma radiation within the energy range of 70-890 keV and having an energy resolution of ~30%. Results. Here, we summarize the system design of the GRBAlpha mission, including the electronics and software components of the detector, some aspects of the platform as well as the current way of semi-autonomous operations. In addition, details are given about the raw data products and telemetry in order to encourage the community for expansion of the receiver network for our initiatives with GRBAlpha and related experiments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10107-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10107) | **A sample of Fast Radio Bursts discovered and localised with MeerTRAP at  the MeerKAT telescope**  |
|| F. Jankowski, et al. -- incl., <mark>M. Kramer</mark> |
|*Appeared on*| *2023-02-21*|
|*Comments*| *23 pages, 10 figures, 3 tables. Submitted to MNRAS in 2022, past first review. Supplementary data are available on Zenodo (this https URL) and via the Transient Name Server*|
|**Abstract**| We present a sample of well-localised Fast Radio Bursts (FRBs) discovered by the MeerTRAP project at the MeerKAT telescope in South Africa. We discovered the three FRBs in single coherent tied-array beams and localised them to an area of ~1 arcmin$^2$. We investigate their burst properties, scattering, repetition rates, and localisations in a multi-wavelength context. FRB 20201211A shows hints of scatter broadening but is otherwise consistent with instrumental dispersion smearing. For FRB 20210202D, we discovered a faint post-cursor burst separated by ~200 ms, suggesting a distinct burst component or a repeat pulse. We attempt to associate the FRBs with host galaxy candidates. For FRB 20210408H, we tentatively (0.35 - 0.53 probability) identify a compatible host at a redshift ~0.5. Additionally, we analyse the MeerTRAP survey properties, such as the survey coverage, fluence completeness, and their implications for the FRB population. Based on the entire sample of 11 MeerTRAP FRBs discovered by the end of 2021, we estimate the FRB all-sky rates and their scaling with the fluence threshold. The inferred FRB all-sky rates at 1.28 GHz are $4.4_{-2.5}^{+4.3}$ and $2.1_{-1.1}^{+1.8} \times 10^3$ sky$^{-1}$ d$^{-1}$ above 0.66 and 3.44 Jy ms for the coherent and incoherent surveys, respectively. The scaling between the MeerTRAP rates is flatter than at higher fluences at the 95 per cent confidence level. There seems to be a deficit of low-fluence FRBs, suggesting a break or turn-over in the rate versus fluence relation below 2 Jy ms. We speculate on cosmological or progenitor-intrinsic origins. The cumulative source counts within our surveys appear consistent with the Euclidean scaling. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.09082-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.09082) | **Jupiter's interior from Juno: Equation-of-state uncertainties and dilute  core extent**  |
|| S. Howard, et al. -- incl., <mark>R. Helled</mark> |
|*Appeared on*| *2023-02-21*|
|*Comments*| **|
|**Abstract**| The Juno mission has provided measurements of Jupiter s gravity field with an outstanding level of accuracy, leading to better constraints on the interior of the planet. Improving our knowledge of the internal structure of Jupiter is key to understanding its formation and evolution but is also important in the framework of exoplanet exploration. In this study, we investigated the differences between the state-of-the-art equations of state and their impact on the properties of interior models. Accounting for uncertainty on the hydrogen and helium equation of state, we assessed the span of the interior features of Jupiter. We carried out an extensive exploration of the parameter space and studied a wide range of interior models using Markov chain Monte Carlo (MCMC) simulations. To consider the uncertainty on the equation of state, we allowed for modifications of the equation of state in our calculations. Our models harbour a dilute core and indicate that Jupiter s internal entropy is higher than what is usually assumed from the Galileo probe measurements. We obtain solutions with extended dilute cores, but contrary to other recent interior models of Jupiter, we also obtain models with small dilute cores. The dilute cores in such solutions extend to 20% of Jupiter s mass, leading to better agreement with formation evolution models. We conclude that the equations of state used in Jupiter models have a crucial effect on the inferred structure and composition. Further explorations of the behaviour of hydrogen helium mixtures at the pressure and temperature conditions in Jupiter will help to constrain the interior of the planet, and therefore its origin. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.09372-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.09372) | **Euclid preparation. XXXI. Performance assessment of the NISP Red-Grism  through spectroscopic simulations for the Wide and Deep surveys**  |
|| Euclid Collaboration, et al. -- incl., <mark>M. Schirmer</mark> |
|*Appeared on*| *2023-02-21*|
|*Comments*| *23 pages, 21 figures*|
|**Abstract**| This work focuses on the pilot run of a simulation campaign aimed at investigating the spectroscopic capabilities of the Euclid Near-Infrared Spectrometer and Photometer (NISP), in terms of continuum and emission line detection in the context of galaxy evolutionary studies. To this purpose we constructed, emulated, and analysed the spectra of 4992 star-forming galaxies at 0.3 <= z <= 2 using the NISP pixel-level simulator. We built the spectral library starting from public multi-wavelength galaxy catalogues, with value-added information on spectral energy distribution (SED) fitting results, and from Bruzual and Charlot (2003) stellar population templates. Rest-frame optical and near-IR nebular emission lines were included using empirical and theoretical relations. We inferred the 3.5 sigma NISP red grism spectroscopic detection limit of the continuum measured in the H band for star-forming galaxies with a median disk half-light radius of 0.4 arcsecond at magnitude H = 19.5 +/- 0.2 AB mag for the EuclidWide Survey and at H = 20.8 +/- 0.6 AB mag for the Euclid Deep Survey. We found a very good agreement with the red grism emission line detection limit requirement for the Wide and Deep surveys. We characterised the effect of the galaxy shape on the detection capability of the red grism and highlighted the degradation of the quality of the extracted spectra as the disk size increases. In particular, we found that the extracted emission line signal to noise ratio (SNR) drops by approx. 45% when the disk size ranges from 0.25 to 1 arcsecond. These trends lead to a correlation between the emission line SNR and the stellar mass of the galaxy and we demonstrate the effect in a stacking analysis that can unveil emission lines otherwise too faint to detect. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure figures/SNR_Els.pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.09987-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.09987) | **Mid-infrared spectroscopy of planetary analogs: A database for planetary  remote sensing**  |
|| A. Morlok, et al. -- incl., <mark>J. Helbert</mark> |
|*Appeared on*| *2023-02-21*|
|*Comments*| **|
|**Abstract**| The MERTIS (MErcury Radiometer and Thermal Infrared Spectrometer) instrument onboard the ESA/JAXA BepiColombo mission will provide mid-infrared data, which will be crucial to characterize the surface mineralogy of Mercury. In order to interpret the results, we are creating a database of mid infrared spectra. As part of a study of synthetic glasses which are to serve as analog materials for the interpretation of remote sensing and modeling data, we present mid infrared data for analog materials of Mercury regolith, surface and mantle compositions. In addition, we provide data for similar analogs of Earth, Moon, Venus, and Mars rocks for a coherent picture. The analog samples have been first characterized by optical microscopy, Raman spectroscopy and EMPA. Powdered size fractions (0-25 micron, 25-63 micron, 63-125 micron, and 125 -250 micron) were studied in reflectance in the mid-infrared range from 2.5 to 18 micron (550 to 2000cm-1), additional micro FTIR analyses were also obtained. Results for the size fractions of the surface and regolith analogs for Mercury show typical features for amorphous material with Christiansen Features (CF) at 8 to 8.1 micron, Reststrahlen Bands (RB) at 9.8 to 9.9 micron, and the Transparency Feature (TF) at 12 micron. The six bulk silicate Mercury analogs have varying CF positions from 8.1 to 9 micron, with RB crystalline features of various olivines dominating in most samples. Similarly, bulk silicate analogs of the other planetary bodies show glassy features for the surface analogs with CF from 7.9 micron (Earth Continental Crust) to 8.3 micron (Lunar Mare), strong RB from 9.5 micron (Earth Continental Crust ) to 10.6 micron (Lunar Mare and Highlands). TF are usually very weak for the glassy analogs. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.10008.md
    + _build/html/tmp_2302.10008/./TOI3235-hs.png
    + _build/html/tmp_2302.10008/./TOI3235-lc.png
    + _build/html/tmp_2302.10008/./mass_radius_period_radius_mass_starmass_061_5205.png
    + _build/html/tmp_2302.10008/./TOI3235-rv.png
    + _build/html/tmp_2302.10008/./TOI3235-iso-gk-gabs-isofeh-SED.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\forloop}[5][1]$
$\newcommand{\tbd}[1]{{\par\bf\textsc{TBD: #1\\}}}$
$\newcommand{\ctbd}[1]$
$\newcommand{\cor}{\textcolor{red}{(corr?) }}$
$\newcommand{\spl}{\textcolor{red}{(spl?) }}$
$\newcommand{\ii}{\'\i }$
$\newcommand{\oo}{\H{o}}$
$\newcommand{\uu}{\H u}$
$\newcommand{\lc}{light curve}$
$\newcommand{\lcs}{light curves}$
$\newcommand{\Lc}{Light curve}$
$\newcommand{\Lcs}{Light curves}$
$\newcommand{\avg}[1]{\ensuremath{\langle #1\rangle}}$
$\newcommand{\med}[1]{\ensuremath{\langle #1\rangle_{med}}}$
$\newcommand{\dpt}{data-point}$
$\newcommand{\dpts}{data-points}$
$\newcommand{\tel}{telescope}$
$\newcommand{\magn}{magnitude}$
$\newcommand{\stan}{standard}$
$\newcommand{\aper}{aperture}$
$\newcommand{\oot}{out-of-transit}$
$\newcommand{\OOT}{Out-of-Transit}$
$\newcommand{\cfa}{Harvard-Smithsonian Center for Astrophysics (CfA)}$
$\newcommand{\cfadigi}{CfA Speedometers}$
$\newcommand{\cmd}{color-magnitude diagram}$
$\newcommand{\conc}[1]{\noindent\par{\noindent{\mathbf \Longrightarrow \bf #1}}}$
$\newcommand{\diam}{\ensuremath{\oslash}}$
$\newcommand{\ccdsize}[1]{\ensuremath{\rm #1\times\rm#1}}$
$\newcommand{\fovsize}[2]{\ensuremath{\rm #1 #2\times\rm#1 #2}}$
$\newcommand{\tsize}[1]{\mbox{\rm #1 m}}$
$\newcommand{\band}[1]{\ensuremath{#1}~band}$
$\newcommand{\ordo}{\ensuremath{\mathcal{O}}}$
$\newcommand{\chisq}{\ensuremath{\chi^2}}$
$\newcommand{\RA}[3]{\ensuremath{#1^{\mathrm h}#2^{\mathrm m}#3^{\mathrm s}}}$
$\newcommand{\DEC}[3]{\ensuremath{#1^{\mathrm d}#2^{\mathrm m}#3^{\mathrm s}}}$
$\newcommand{\ghr}{\ensuremath{^h}}$
$\newcommand{\gmin}{\ensuremath{^m}}$
$\newcommand{\Ks}{\ensuremath{K_s}}$
$\newcommand{\masy}{\ensuremath{\rm mas yr^{-1}}}$
$\newcommand{\kms}{\ensuremath{\rm km s^{-1}}}$
$\newcommand{\ms}{\ensuremath{\rm m s^{-1}}}$
$\newcommand{\msd}{\ensuremath{\rm m s^{-1} d^{-1}}}$
$\newcommand{\mss}{\ensuremath{\rm m s^{-2}}}$
$\newcommand{\gcmc}{\ensuremath{\rm g cm^{-3}}}$
$\newcommand{\ergscmsq}{\ensuremath{\rm erg s^{-1} cm^{-2}}}$
$\newcommand{\C}{\ensuremath{^{\circ}C\;}}$
$\newcommand{\el}{\ensuremath{e^-}}$
$\newcommand{\sqarcsec}{\ensuremath{\Box^{\prime\prime}}}$
$\newcommand{\sqarcdeg}{\ensuremath{\Box^{\circ}}}$
$\newcommand{\pxs}{\ensuremath{\rm \arcsec pixel^{-1}}}$
$\newcommand{\aduel}{\ensuremath{\lbrack ADU/\el \rbrack}}$
$\newcommand{\eladu}{\ensuremath{\lbrack \el/ADU \rbrack}}$
$\newcommand{\adupixs}{\ensuremath{\rm ADU/(pix  s)}}$
$\newcommand{\elpixs}{\ensuremath{\rm \el/(pix  s)}}$
$\newcommand{\masyr}{\ensuremath{\rm mas yr^{-1}}}$
$\newcommand{\msini}{\ensuremath{m \sin i}}$
$\newcommand{\mplsini}{\ensuremath{\mpl\sin i}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\logg}{\ensuremath{\log{g}}}$
$\newcommand{\vsini}{\ensuremath{v \sin{i}}}$
$\newcommand{\feh}{\ensuremath{\rm[Fe/H]}}$
$\newcommand{\logl}{\ensuremath{\log{L}}}$
$\newcommand{\vmac}{\ensuremath{v_{\rm mac}}}$
$\newcommand{\vmic}{\ensuremath{v_{\rm mic}}}$
$\newcommand{\rhk}{\ensuremath{R^{\prime}_{\rm HK}}}$
$\newcommand{\logrhk}{\ensuremath{\log\rhk}}$
$\newcommand{\Savg}{\ensuremath{\langle S\rangle}}$
$\newcommand{\vic}{\ensuremath{V\!-\!I_C}}$
$\newcommand{\ebv}{\ensuremath{E(B\!-\!V)}}$
$\newcommand{\rsun}{\ensuremath{R_\sun}}$
$\newcommand{\msun}{\ensuremath{M_\sun}}$
$\newcommand{\lsun}{\ensuremath{L_\sun}}$
$\newcommand{\loglsun}{\ensuremath{\log{L_\sun}}}$
$\newcommand{\teffsun}{\ensuremath{T_{eff,\sun}}}$
$\newcommand{\rhosun}{\ensuremath{\rho_\sun}}$
$\newcommand{\loggsun}{\ensuremath{\log{g_{\sun}}}}$
$\newcommand{\rstar}{\ensuremath{R_\star}}$
$\newcommand{\mstar}{\ensuremath{M_\star}}$
$\newcommand{\lstar}{\ensuremath{L_\star}}$
$\newcommand{\astar}{\ensuremath{a_\star}}$
$\newcommand{\loglstar}{\ensuremath{\log{L_\star}}}$
$\newcommand{\teffstar}{\ensuremath{T_{\rm eff\star}}}$
$\newcommand{\rhostar}{\ensuremath{\rho_\star}}$
$\newcommand{\loggstar}{\ensuremath{\log{g_{\star}}}}$
$\newcommand{\rearth}{\ensuremath{R_\earth}}$
$\newcommand{\mearth}{\ensuremath{M_\earth}}$
$\newcommand{\learth}{\ensuremath{L_\earth}}$
$\newcommand{\teffearth}{\ensuremath{T_{\rm eff,\earth}}}$
$\newcommand{\rhoearth}{\ensuremath{\rho_\earth}}$
$\newcommand{\rpl}{\ensuremath{R_{p}}}$
$\newcommand{\mpl}{\ensuremath{M_{p}}}$
$\newcommand{\lpl}{\ensuremath{L_{p}}}$
$\newcommand{\teffpl}{\ensuremath{T_{\rm eff,{p}}}}$
$\newcommand{\rhopl}{\ensuremath{\rho_{p}}}$
$\newcommand{\ipl}{\ensuremath{i_{p}}}$
$\newcommand{\epl}{\ensuremath{e_{p}}}$
$\newcommand{\gpl}{\ensuremath{g_{p}}}$
$\newcommand{\loggpl}{\ensuremath{\log g_{p}}}$
$\newcommand{\arstar}{\ensuremath{a/\rstar}}$
$\newcommand{\zrstar}{\ensuremath{\zeta/\rstar}}$
$\newcommand{\rjup}{\ensuremath{R_{\rm J}}}$
$\newcommand{\mjup}{\ensuremath{M_{\rm J}}}$
$\newcommand{\ljup}{\ensuremath{L_{\rm J}}}$
$\newcommand{\teffjup}{\ensuremath{T_{eff,{\rm J}}}}$
$\newcommand{\rhojup}{\ensuremath{\rho_{\rm J}}}$
$\newcommand{\gjup}{\ensuremath{\g_{\rm J}}}$
$\newcommand{\rjuplong}{\ensuremath{R_{\rm Jup}}}$
$\newcommand{\mjuplong}{\ensuremath{M_{\rm Jup}}}$
$\newcommand{\ljuplong}{\ensuremath{L_{\rm Jup}}}$
$\newcommand{\teffjuplong}{\ensuremath{T_{eff,{\rm Jup}}}}$
$\newcommand{\rhojuplong}{\ensuremath{\rho_{\rm Jup}}}$
$\newcommand{\gjuplong}{\ensuremath{\g_{\rm Jup}}}$
$\newcommand{\pack}[1]{\textsc{\lowercase{#1}}}$
$\newcommand{\prog}[1]{\texttt{\lowercase{#1}}}$
$\newcommand{\iraf}{\pack{iraf}}$
$\newcommand{\todcor}{\prog{todcor}}$
$\newcommand{\xcsao}{\prog{xcsao}}$
$\newcommand{\daophot}{\pack{daophot}}$
$\newcommand{\fihat}{\pack{fihat}}$
$\newcommand{\fistar}{\prog{fistar}}$
$\newcommand{\fiphot}{\prog{fiphot}}$
$\newcommand{\grmatch}{\prog{grmatch}}$
$\newcommand{\grtrans}{\prog{grtrans}}$
$\newcommand{\refp}[1]{p.~\pageref{#1}}$
$\newcommand{\reffig}[1]{Fig.~\ref{fig:#1}}$
$\newcommand{\refsec}[1]{\mbox{\S \ref{sec:#1}}}$
$\newcommand{\refeq}[1]{Eq.~\ref{eq:#1}}$
$\newcommand{\reftab}[1]{Tab.~\ref{tab:#1}}$
$\newcommand{\reffigl}[1]{Figure~\ref{fig:#1}}$
$\newcommand{\refsecl}[1]{\mbox{Section \ref{sec:#1}}}$
$\newcommand{\refeql}[1]{Equation~\ref{eq:#1}}$
$\newcommand{\reftabl}[1]{Table~\ref{tab:#1}}$
$\newcommand{\reffigp}[1]{\reffig{#1} on \pref{fig:#1}}$
$\newcommand{\refsecp}[1]{\refsec{#1} on \pref{sec:#1}}$
$\newcommand{\refeqp}[1]{\refeq{#1} on \pref{eq:#1}}$
$\newcommand{\reftabp}[1]{\reftab{#1} on \pref{tab:#1}}$
$\newcommand{\reffigls}[2]{Figures~\ref{fig:#1}-\ref{fig:#2}}$
$\newcommand{\reftabls}[2]{Tables~\ref{tab:#1}-\ref{tab:#2}}$
$\newcommand{\flwof}{\mbox{FLWO 1.2 m}}$
$\newcommand{\flwos}{\mbox{FLWO 1.5 m}}$
$\newcommand{\flwot}{\mbox{TopHAT 0.25 m}}$
$\newcommand{\mmt}{\mbox{MMT 6.5 m}}$
$\newcommand{\ssts}{{\em Spitzer}}$
$\newcommand{\sstL}{{\em Spitzer Space Telescope}}$
$\newcommand{\hst}{{\em HST}}$
$\newcommand{\wom}{\mbox{Wise 1 m}}$
$\newcommand{\piszkessch}{Konkoly 0.6 m Schmidt}$
$\newcommand{\piszkesrcc}{Konkoly 1 m RCC}$
$\newcommand{\dscu}{\mbox{\delta Scuti}}$
$\newcommand{\gdor}{\mbox{\gamma Dor}}$
$\newcommand{\hj}{hot Jupiter}$
$\newcommand{\vhj}{very hot Jupiter}$
$\newcommand{\hd}[1]{\mbox{HD #1}}$
$\newcommand{\BD}[1]{\mbox{BD #1}}$
$\newcommand{\hip}[1]{\mbox{HIP #1}}$
$\newcommand{\gj}[1]{\mbox{GJ #1}}$
$\newcommand{\tbn}[1]{\tablenotemark{#1}}$
$\newcommand{\hatcurhtr}{TOI-3235}$
$\newcommand{\hatcurfield}{\ensuremath{string}}$
$\newcommand{\hatcurCCra}{\ensuremath{13^{\mathrm h}49^{\mathrm m}53.9777{\mathrm s}}}$
$\newcommand{\hatcurCCdec}{\ensuremath{-46{\arcdeg}03{\arcmin}58.4541{\arcsec}}}$
$\newcommand{\hatcurCCmag}{NULL}$
$\newcommand{\hatcurCCtwomass}{2MASS~13495398-4603583}$
$\newcommand{\hatcurCCgsc}{GSC~NULL}$
$\newcommand{\hatcurCCgaia}{GAIA~6107144260247338240}$
$\newcommand{\hatcurCCgaiadrtwo}{GAIA~DR2~6107144260251920000}$
$\newcommand{\hatcurCCgaiadrthree}{GAIA~DR3~6107144260251920000}$
$\newcommand{\hatcurTICID}{TIC~243641947}$
$\newcommand{\hatcurCCtassmv}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmvshort}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmB}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmBshort}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmI}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmIshort}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmg}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmgshort}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmr}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmrshort}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmi}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmishort}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCparallax}{\ensuremath{13.781\pm0.027}}$
$\newcommand{\hatcurCCgaiamG}{\ensuremath{14.4605\pm0.0028}}$
$\newcommand{\hatcurCCgaiamBP}{\ensuremath{15.9000\pm0.0042}}$
$\newcommand{\hatcurCCgaiamRP}{\ensuremath{13.2881\pm0.0038}}$
$\newcommand{\hatcurCCgaiamGthree}{\ensuremath{14.4605\pm0.0028}}$
$\newcommand{\hatcurCCgaiamBPthree}{\ensuremath{15.9000\pm0.0042}}$
$\newcommand{\hatcurCCgaiamRPthree}{\ensuremath{13.2881\pm0.0038}}$
$\newcommand{\hatcurCCtwomassJmag}{\ensuremath{11.706\pm0.025}}$
$\newcommand{\hatcurCCtwomassHmag}{\ensuremath{11.099\pm0.024}}$
$\newcommand{\hatcurCCtwomassKmag}{\ensuremath{10.819\pm0.021}}$
$\newcommand{\hatcurCCcitJmag}{\ensuremath{11.695\pm0.026}}$
$\newcommand{\hatcurCCcitHmag}{\ensuremath{11.088\pm0.026}}$
$\newcommand{\hatcurCCcitKmag}{\ensuremath{10.843\pm0.022}}$
$\newcommand{\hatcurCCbbJmag}{\ensuremath{11.787\pm0.029}}$
$\newcommand{\hatcurCCbbHmag}{\ensuremath{11.116\pm0.026}}$
$\newcommand{\hatcurCCbbKmag}{\ensuremath{10.863\pm0.022}}$
$\newcommand{\hatcurCCesoJmag}{\ensuremath{11.796\pm0.033}}$
$\newcommand{\hatcurCCesoHmag}{\ensuremath{11.118\pm0.047}}$
$\newcommand{\hatcurCCesoKmag}{\ensuremath{10.859\pm0.024}}$
$\newcommand{\hatcurCCesoJHmag}{\ensuremath{0.678\pm0.054}}$
$\newcommand{\hatcurCCesoJKmag}{\ensuremath{0.936\pm0.038}}$
$\newcommand{\hatcurCCesoHKmag}{\ensuremath{0.258\pm0.051}}$
$\newcommand{\hatcurCCWonemag}{\ensuremath{10.694\pm0.023}}$
$\newcommand{\hatcurCCWtwomag}{\ensuremath{10.548\pm0.021}}$
$\newcommand{\hatcurCCWthreemag}{\ensuremath{10.458\pm0.064}}$
$\newcommand{\hatcurCCWfourmag}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurLCdip}{\ensuremath{0.1}}$
$\newcommand{\hatcurLCrprstar}{\ensuremath{0.2828\pm0.0016}}$
$\newcommand{\hatcurLCbsq}{\ensuremath{0.261_{-0.012}^{+0.012}}}$
$\newcommand{\hatcurLCimp}{\ensuremath{0.511_{-0.012}^{+0.011}}}$
$\newcommand{\hatcurLCzeta}{\ensuremath{44.41\pm0.32}}$
$\newcommand{\hatcurLCdur}{\ensuremath{0.06165\pm0.00021}}$
$\newcommand{\hatcurLCdurshort}{\ensuremath{0.0616}}$
$\newcommand{\hatcurLCdurhr}{\ensuremath{1.4795\pm0.0049}}$
$\newcommand{\hatcurLCdurhrshort}{\ensuremath{1.480}}$
$\newcommand{\hatcurLCq}{\ensuremath{0.023800\pm0.000086}}$
$\newcommand{\hatcurLCqshort}{\ensuremath{0.024}}$
$\newcommand{\hatcurLCingdur}{\ensuremath{0.01765\pm0.00030}}$
$\newcommand{\hatcurLCP}{\ensuremath{2.59261842\pm0.00000041}}$
$\newcommand{\hatcurLCPprec}{\ensuremath{2.5926184}}$
$\newcommand{\hatcurLCPshort}{\ensuremath{2.5926}}$
$\newcommand{\hatcurLCT}{\ensuremath{2459690.001730\pm0.000045}}$
$\newcommand{\hatcurLCTA}{\ensuremath{2457797.39028\pm0.00028}}$
$\newcommand{\hatcurLCTB}{\ensuremath{2459762.595048\pm0.000053}}$
$\newcommand{\hatcurLChatnetmA}{\ensuremath{13.62811\pm0.00072}}$
$\newcommand{\hatcurLCiblendA}{\ensuremath{0.982\pm0.022}}$
$\newcommand{\hatcurLChatnetmB}{\ensuremath{13.63138\pm0.00053}}$
$\newcommand{\hatcurLCiblendB}{\ensuremath{0.957\pm0.027}}$
$\newcommand{\hatcurLChatnetmC}{\ensuremath{-7.03640\pm0.00011}}$
$\newcommand{\hatcurLCiblendC}{\ensuremath{1.084\pm0.011}}$
$\newcommand{\hatcurLChatnetmD}{\ensuremath{-6.996030\pm0.000084}}$
$\newcommand{\hatcurLCiblendD}{\ensuremath{1.1248\pm0.0088}}$
$\newcommand{\hatcurLCrho}{\ensuremath{10.99\pm0.13}}$
$\newcommand{\hatcurSMEiteff}{\ensuremath{3196\pm67}}$
$\newcommand{\hatcurSMEizfeh}{\ensuremath{-0.02\pm0.10}}$
$\newcommand{\hatcurSMEizfehshort}{\ensuremath{-0.02}}$
$\newcommand{\hatcurSMEilogg}{\ensuremath{4.80\pm0.50}}$
$\newcommand{\hatcurSMEivsin}{\ensuremath{0\pm50}}$
$\newcommand{\hatcurSMEivmac}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurSMEivmic}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurextraerrMJ}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMJtwosiglim}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMH}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMHtwosiglim}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMKs}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMKstwosiglim}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMG}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMGtwosiglim}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMRP}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMRPtwosiglim}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMWone}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMWonetwosiglim}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMWtwo}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMWtwotwosiglim}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurLBiB}{\ensuremath{0.5035}}$
$\newcommand{\hatcurLBiiB}{\ensuremath{0.3376}}$
$\newcommand{\hatcurLBiV}{\ensuremath{0.4840}}$
$\newcommand{\hatcurLBiiV}{\ensuremath{0.3559}}$
$\newcommand{\hatcurLBiR}{\ensuremath{0.4142}}$
$\newcommand{\hatcurLBiiR}{\ensuremath{0.3369}}$
$\newcommand{\hatcurLBiI}{\ensuremath{0.2042}}$
$\newcommand{\hatcurLBiiI}{\ensuremath{0.3912}}$
$\newcommand{\hatcurLBiu}{\ensuremath{0.4328}}$
$\newcommand{\hatcurLBiiu}{\ensuremath{0.3416}}$
$\newcommand{\hatcurLBig}{\ensuremath{0.31\pm0.12}}$
$\newcommand{\hatcurLBiig}{\ensuremath{0.35\pm0.14}}$
$\newcommand{\hatcurLBir}{\ensuremath{0.41\pm0.15}}$
$\newcommand{\hatcurLBiir}{\ensuremath{0.32\pm0.16}}$
$\newcommand{\hatcurLBii}{\ensuremath{0.30\pm0.11}}$
$\newcommand{\hatcurLBiii}{\ensuremath{0.26_{-0.15}^{+0.11}}}$
$\newcommand{\hatcurLBiz}{\ensuremath{0.140_{-0.081}^{+0.106}}}$
$\newcommand{\hatcurLBiiz}{\ensuremath{0.17\pm0.13}}$
$\newcommand{\hatcurLBiJ}{\ensuremath{0.126\pm0.072}}$
$\newcommand{\hatcurLBiiJ}{\ensuremath{0.12\pm0.12}}$
$\newcommand{\hatcurLBiH}{\ensuremath{0.0889}}$
$\newcommand{\hatcurLBiiH}{\ensuremath{0.2815}}$
$\newcommand{\hatcurLBiK}{\ensuremath{0.0827}}$
$\newcommand{\hatcurLBiiK}{\ensuremath{0.2458}}$
$\newcommand{\hatcurLBiT}{\ensuremath{0.24\pm0.12}}$
$\newcommand{\hatcurLBiiT}{\ensuremath{0.33\pm0.16}}$
$\newcommand{\hatcurLBikep}{\ensuremath{0.2909}}$
$\newcommand{\hatcurLBiikep}{\ensuremath{0.4272}}$
$\newcommand{\hatcurLBiC}{\ensuremath{0.2393}}$
$\newcommand{\hatcurLBiiC}{\ensuremath{0.4323}}$
$\newcommand{\hatcurLBiM}{\ensuremath{0.4020}}$
$\newcommand{\hatcurLBiiM}{\ensuremath{0.4034}}$
$\newcommand{\hatcurLBiSone}{\ensuremath{0.0655}}$
$\newcommand{\hatcurLBiiSone}{\ensuremath{0.1827}}$
$\newcommand{\hatcurLBiStwo}{\ensuremath{0.0480}}$
$\newcommand{\hatcurLBiiStwo}{\ensuremath{0.1526}}$
$\newcommand{\hatcurLBiSthree}{\ensuremath{0.0530}}$
$\newcommand{\hatcurLBiiSthree}{\ensuremath{0.1348}}$
$\newcommand{\hatcurLBiSfour}{\ensuremath{0.0565}}$
$\newcommand{\hatcurLBiiSfour}{\ensuremath{0.1029}}$
$\newcommand{\hatcurISOm}{\ensuremath{0.3939\pm0.0030}}$
$\newcommand{\hatcurISOmshort}{\ensuremath{0.39}}$
$\newcommand{\hatcurISOmlong}{\ensuremath{0.3939\pm0.0030}}$
$\newcommand{\hatcurISOr}{\ensuremath{0.3697\pm0.0018}}$
$\newcommand{\hatcurISOrshort}{\ensuremath{0.37}}$
$\newcommand{\hatcurISOrlong}{\ensuremath{0.3697\pm0.0018}}$
$\newcommand{\hatcurISOrho}{\ensuremath{10.99\pm0.13}}$
$\newcommand{\hatcurISOrholong}{\ensuremath{10.99\pm0.13}}$
$\newcommand{\hatcurISOlogg}{\ensuremath{4.8976\pm0.0035}}$
$\newcommand{\hatcurISOlum}{\ensuremath{0.01623\pm0.00018}}$
$\newcommand{\hatcurISOlumshort}{\ensuremath{0.02}}$
$\newcommand{\hatcurISOteff}{\ensuremath{3388.8\pm5.9}}$
$\newcommand{\hatcurISOteffshort}{\ensuremath{3389}}$
$\newcommand{\hatcurISOzfeh}{\ensuremath{0.264_{-0.017}^{+0.013}}}$
$\newcommand{\hatcurISOage}{\ensuremath{0.394_{-0.090}^{+0.152}}}$
$\newcommand{\hatcurISOspec}{M}$
$\newcommand{\hatcurRVK}{\ensuremath{182.9\pm3.3}}$
$\newcommand{\hatcurRVKtwosiglim}{\ensuremath{<188.1}}$
$\newcommand{\hatcurRVrk}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurRVrh}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurRVk}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurRVh}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurRVtrone}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurRVtrtwo}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurRVgamma}{\ensuremath{-12182.3\pm2.4}}$
$\newcommand{\hatcurRVjitter}{\ensuremath{0.1\pm4.2}}$
$\newcommand{\hatcurRVjittertwosiglim}{\ensuremath{<9.6}}$
$\newcommand{\hatcurRVfitrms}{\ensuremath{.1fym}}$
$\newcommand{\hatcurRVeccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurRVeccentwosiglim}{\ensuremath{<0.000}}$
$\newcommand{\hatcurRVomega}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurPPi}{\ensuremath{88.140\pm0.046}}$
$\newcommand{\hatcurPPg}{\ensuremath{15.90\pm0.37}}$
$\newcommand{\hatcurPPloggold}{\ensuremath{3.202\pm0.010}}$
$\newcommand{\hatcurPPlogg}{\ensuremath{3.202\pm0.041}}$
$\newcommand{\hatcurPParold}{\ensuremath{15.754\pm0.062}}$
$\newcommand{\hatcurPPar}{\ensuremath{15.75\pm0.73}}$
$\newcommand{\hatcurPParelold}{\ensuremath{0.027089\pm0.000069}}$
$\newcommand{\hatcurPParel}{\ensuremath{0.02709\pm0.00046}}$
$\newcommand{\hatcurPPrhoold}{\ensuremath{0.783\pm0.022}}$
$\newcommand{\hatcurPPrho}{\ensuremath{0.78\pm0.11}}$
$\newcommand{\hatcurPPmold}{\ensuremath{0.665\pm0.013}}$
$\newcommand{\hatcurPPm}{\ensuremath{0.665\pm0.025}}$
$\newcommand{\hatcurPPmtwosiglim}{\ensuremath{<0.68}}$
$\newcommand{\hatcurPPmshort}{\ensuremath{0.66}}$
$\newcommand{\hatcurPPmlong}{\ensuremath{0.665\pm0.025}}$
$\newcommand{\hatcurPPme}{\ensuremath{211.3\pm4.0}}$
$\newcommand{\hatcurPPmeshort}{\ensuremath{211.3}}$
$\newcommand{\hatcurPPmelong}{\ensuremath{211.3\pm4.0}}$
$\newcommand{\hatcurPProld}{\ensuremath{1.0174\pm0.0078}}$
$\newcommand{\hatcurPPr}{\ensuremath{1.017\pm0.044}}$
$\newcommand{\hatcurPPrshort}{\ensuremath{1.02}}$
$\newcommand{\hatcurPPrlong}{\ensuremath{1.017\pm0.044}}$
$\newcommand{\hatcurPPre}{\ensuremath{11.404\pm0.087}}$
$\newcommand{\hatcurPPreshort}{\ensuremath{11.4}}$
$\newcommand{\hatcurPPrelong}{\ensuremath{11.404\pm0.087}}$
$\newcommand{\hatcurPPmrcorr}{\ensuremath{0.09}}$
$\newcommand{\hatcurPPteffold}{\ensuremath{603.8\pm1.2}}$
$\newcommand{\hatcurPPteff}{\ensuremath{604\pm19}}$
$\newcommand{\hatcurPPteffshort}{\ensuremath{604}}$
$\newcommand{\hatcurPPthetaold}{\ensuremath{0.0896\pm0.0018}}$
$\newcommand{\hatcurPPtheta}{\ensuremath{0.0896\pm0.0042}}$
$\newcommand{\hatcurPPfluxperi}{\ensuremath{3.014\pm0.024}}$
$\newcommand{\hatcurPPfluxperidim}{\ensuremath{7}}$
$\newcommand{\hatcurPPfluxap}{\ensuremath{3.014\pm0.024}}$
$\newcommand{\hatcurPPfluxapdim}{\ensuremath{7}}$
$\newcommand{\hatcurPPfluxavg}{\ensuremath{3.014\pm0.024}}$
$\newcommand{\hatcurPPfluxavgdim}{\ensuremath{7}}$
$\newcommand{\hatcurPPfluxavglogold}{\ensuremath{7.4791\pm0.0035}}$
$\newcommand{\hatcurPPfluxavglog}{\ensuremath{7.479\pm0.018}}$
$\newcommand{\hatcurXsecphase}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurXsecondary}{\ensuremath{2459691.298040\pm0.000045}}$
$\newcommand{\hatcurXsecdur}{\ensuremath{0.06165\pm0.00021}}$
$\newcommand{\hatcurXsecingdur}{\ensuremath{0.01765\pm0.00030}}$
$\newcommand{\hatcurPPphiconj}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurPPperi}{\ensuremath{2459689.353577\pm0.000045}}$
$\newcommand{\hatcurPPaequiv}{\ensuremath{0.21250\pm0.00085}}$
$\newcommand{\hatcurPPtcirc}{\ensuremath{62.0\pm2.5}}$
$\newcommand{\hatcurPPtinfall}{\ensuremath{14150\pm370}}$
$\newcommand{\hatcurXdist}{\ensuremath{72.50\pm0.13}}$
$\newcommand{\hatcurXAv}{\ensuremath{0.064\pm0.021}}$
$\newcommand{\hatcurXdistred}{\ensuremath{72.50\pm0.12}}$
$\newcommand{\hatcurXEBV}{\ensuremath{0.0210_{-0.0070}^{+0.0050}}}$
$\newcommand{\hatcurCCpmra}{\ensuremath{-170.503\pm0.028}}$
$\newcommand{\hatcurCCpmdec}{\ensuremath{-64.264\pm0.023}}$
$\newcommand{\hatcurCCpm}{\ensuremath{182.212\pm0.036}}$
$\newcommand{\hatcurhtreccen}{TOI3235}$
$\newcommand{\hatcurfieldeccen}{\ensuremath{string}}$
$\newcommand{\hatcurCCraeccen}{\ensuremath{13^{\mathrm h}49^{\mathrm m}53.9777{\mathrm s}}}$
$\newcommand{\hatcurCCdececcen}{\ensuremath{-46{\arcdeg}03{\arcmin}58.4541{\arcsec}}}$
$\newcommand{\hatcurCCmageccen}{NULL}$
$\newcommand{\hatcurCCtwomasseccen}{2MASS~13495398-4603583}$
$\newcommand{\hatcurCCgsceccen}{GSC~NULL}$
$\newcommand{\hatcurCCgaiaeccen}{GAIA~6107144260247338240}$
$\newcommand{\hatcurCCgaiadrtwoeccen}{GAIA~DR2~6107144260251920000}$
$\newcommand{\hatcurCCtassmveccen}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmvshorteccen}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmBeccen}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmBshorteccen}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmIeccen}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmIshorteccen}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmgeccen}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmgshorteccen}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmreccen}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmrshorteccen}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCtassmieccen}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurCCtassmishorteccen}{\ensuremath{0.0}}$
$\newcommand{\hatcurCCparallaxeccen}{\ensuremath{13.781\pm0.027}}$
$\newcommand{\hatcurCCgaiamGeccen}{\ensuremath{14.4605\pm0.0028}}$
$\newcommand{\hatcurCCgaiamBPeccen}{\ensuremath{15.9000\pm0.0042}}$
$\newcommand{\hatcurCCgaiamRPeccen}{\ensuremath{13.2881\pm0.0038}}$
$\newcommand{\hatcurCCtwomassJmageccen}{\ensuremath{11.706\pm0.025}}$
$\newcommand{\hatcurCCtwomassHmageccen}{\ensuremath{11.099\pm0.024}}$
$\newcommand{\hatcurCCtwomassKmageccen}{\ensuremath{10.819\pm0.021}}$
$\newcommand{\hatcurCCcitJmageccen}{\ensuremath{11.695\pm0.026}}$
$\newcommand{\hatcurCCcitHmageccen}{\ensuremath{11.088\pm0.026}}$
$\newcommand{\hatcurCCcitKmageccen}{\ensuremath{10.843\pm0.022}}$
$\newcommand{\hatcurCCbbJmageccen}{\ensuremath{11.787\pm0.029}}$
$\newcommand{\hatcurCCbbHmageccen}{\ensuremath{11.116\pm0.026}}$
$\newcommand{\hatcurCCbbKmageccen}{\ensuremath{10.863\pm0.022}}$
$\newcommand{\hatcurCCesoJmageccen}{\ensuremath{11.796\pm0.033}}$
$\newcommand{\hatcurCCesoHmageccen}{\ensuremath{11.118\pm0.047}}$
$\newcommand{\hatcurCCesoKmageccen}{\ensuremath{10.859\pm0.024}}$
$\newcommand{\hatcurCCesoJHmageccen}{\ensuremath{0.678\pm0.054}}$
$\newcommand{\hatcurCCesoJKmageccen}{\ensuremath{0.936\pm0.038}}$
$\newcommand{\hatcurCCesoHKmageccen}{\ensuremath{0.258\pm0.051}}$
$\newcommand{\hatcurCCWonemageccen}{\ensuremath{10.694\pm0.023}}$
$\newcommand{\hatcurCCWtwomageccen}{\ensuremath{10.548\pm0.021}}$
$\newcommand{\hatcurCCWthreemageccen}{\ensuremath{10.458\pm0.064}}$
$\newcommand{\hatcurCCWfourmageccen}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurLCdipeccen}{\ensuremath{0.1}}$
$\newcommand{\hatcurLCrprstareccen}{\ensuremath{0.2826\pm0.0015}}$
$\newcommand{\hatcurLCbsqeccen}{\ensuremath{0.258_{-0.013}^{+0.012}}}$
$\newcommand{\hatcurLCimpeccen}{\ensuremath{0.507_{-0.013}^{+0.012}}}$
$\newcommand{\hatcurLCzetaeccen}{\ensuremath{44.43\pm0.30}}$
$\newcommand{\hatcurLCdureccen}{\ensuremath{0.06154\pm0.00021}}$
$\newcommand{\hatcurLCdurshorteccen}{\ensuremath{0.0615}}$
$\newcommand{\hatcurLCdurhreccen}{\ensuremath{1.4770\pm0.0051}}$
$\newcommand{\hatcurLCdurhrshorteccen}{\ensuremath{1.477}}$
$\newcommand{\hatcurLCqeccen}{\ensuremath{0.023700\pm0.000096}}$
$\newcommand{\hatcurLCqshorteccen}{\ensuremath{0.024}}$
$\newcommand{\hatcurLCingdureccen}{\ensuremath{0.01753\pm0.00031}}$
$\newcommand{\hatcurLCPeccen}{\ensuremath{2.59261840\pm0.00000040}}$
$\newcommand{\hatcurLCPprececcen}{\ensuremath{2.5926184}}$
$\newcommand{\hatcurLCPshorteccen}{\ensuremath{2.5926}}$
$\newcommand{\hatcurLCTeccen}{\ensuremath{2459684.816480\pm0.000046}}$
$\newcommand{\hatcurLCTAeccen}{\ensuremath{2457797.39029\pm0.00026}}$
$\newcommand{\hatcurLCTBeccen}{\ensuremath{2459762.595037\pm0.000053}}$
$\newcommand{\hatcurLChatnetmAeccen}{\ensuremath{13.62805\pm0.00074}}$
$\newcommand{\hatcurLCiblendAeccen}{\ensuremath{0.981\pm0.022}}$
$\newcommand{\hatcurLChatnetmBeccen}{\ensuremath{13.63141\pm0.00051}}$
$\newcommand{\hatcurLCiblendBeccen}{\ensuremath{0.965\pm0.029}}$
$\newcommand{\hatcurLChatnetmCeccen}{\ensuremath{-7.03641\pm0.00011}}$
$\newcommand{\hatcurLCiblendCeccen}{\ensuremath{1.085\pm0.011}}$
$\newcommand{\hatcurLChatnetmDeccen}{\ensuremath{-6.996010\pm0.000083}}$
$\newcommand{\hatcurLCiblendDeccen}{\ensuremath{1.1257\pm0.0089}}$
$\newcommand{\hatcurLCrhoeccen}{\ensuremath{10.64\pm0.23}}$
$\newcommand{\hatcurSMEiteffeccen}{\ensuremath{3196\pm67}}$
$\newcommand{\hatcurSMEizfeheccen}{\ensuremath{-0.02\pm0.10}}$
$\newcommand{\hatcurSMEizfehshorteccen}{\ensuremath{-0.02}}$
$\newcommand{\hatcurSMEiloggeccen}{\ensuremath{4.80\pm0.50}}$
$\newcommand{\hatcurSMEivsineccen}{\ensuremath{0\pm50}}$
$\newcommand{\hatcurSMEivmaceccen}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurSMEivmiceccen}{\ensuremath{nff\pmnff}}$
$\newcommand{\hatcurextraerrMJeccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMJtwosiglimeccen}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMHeccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMHtwosiglimeccen}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMKseccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMKstwosiglimeccen}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMGeccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMGtwosiglimeccen}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMRPeccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMRPtwosiglimeccen}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMWoneeccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMWonetwosiglimeccen}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurextraerrMWtwoeccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurextraerrMWtwotwosiglimeccen}{\ensuremath{<0.0200}}$
$\newcommand{\hatcurLBiBeccen}{\ensuremath{0.5035}}$
$\newcommand{\hatcurLBiiBeccen}{\ensuremath{0.3376}}$
$\newcommand{\hatcurLBiVeccen}{\ensuremath{0.4840}}$
$\newcommand{\hatcurLBiiVeccen}{\ensuremath{0.3559}}$
$\newcommand{\hatcurLBiReccen}{\ensuremath{0.4142}}$
$\newcommand{\hatcurLBiiReccen}{\ensuremath{0.3369}}$
$\newcommand{\hatcurLBiIeccen}{\ensuremath{0.2042}}$
$\newcommand{\hatcurLBiiIeccen}{\ensuremath{0.3912}}$
$\newcommand{\hatcurLBiueccen}{\ensuremath{0.4328}}$
$\newcommand{\hatcurLBiiueccen}{\ensuremath{0.3416}}$
$\newcommand{\hatcurLBigeccen}{\ensuremath{0.34\pm0.13}}$
$\newcommand{\hatcurLBiigeccen}{\ensuremath{0.31\pm0.16}}$
$\newcommand{\hatcurLBireccen}{\ensuremath{0.39\pm0.15}}$
$\newcommand{\hatcurLBiireccen}{\ensuremath{0.33\pm0.17}}$
$\newcommand{\hatcurLBiieccen}{\ensuremath{0.303\pm0.097}}$
$\newcommand{\hatcurLBiiieccen}{\ensuremath{0.24\pm0.13}}$
$\newcommand{\hatcurLBizeccen}{\ensuremath{0.154\pm0.096}}$
$\newcommand{\hatcurLBiizeccen}{\ensuremath{0.16\pm0.14}}$
$\newcommand{\hatcurLBiJeccen}{\ensuremath{0.128\pm0.082}}$
$\newcommand{\hatcurLBiiJeccen}{\ensuremath{0.12\pm0.13}}$
$\newcommand{\hatcurLBiHeccen}{\ensuremath{0.0889}}$
$\newcommand{\hatcurLBiiHeccen}{\ensuremath{0.2815}}$
$\newcommand{\hatcurLBiKeccen}{\ensuremath{0.0827}}$
$\newcommand{\hatcurLBiiKeccen}{\ensuremath{0.2458}}$
$\newcommand{\hatcurLBiTeccen}{\ensuremath{0.24\pm0.11}}$
$\newcommand{\hatcurLBiiTeccen}{\ensuremath{0.32\pm0.15}}$
$\newcommand{\hatcurLBikepeccen}{\ensuremath{0.2909}}$
$\newcommand{\hatcurLBiikepeccen}{\ensuremath{0.4272}}$
$\newcommand{\hatcurLBiCeccen}{\ensuremath{0.2393}}$
$\newcommand{\hatcurLBiiCeccen}{\ensuremath{0.4323}}$
$\newcommand{\hatcurLBiMeccen}{\ensuremath{0.4020}}$
$\newcommand{\hatcurLBiiMeccen}{\ensuremath{0.4034}}$
$\newcommand{\hatcurLBiSoneeccen}{\ensuremath{0.0655}}$
$\newcommand{\hatcurLBiiSoneeccen}{\ensuremath{0.1827}}$
$\newcommand{\hatcurLBiStwoeccen}{\ensuremath{0.0480}}$
$\newcommand{\hatcurLBiiStwoeccen}{\ensuremath{0.1526}}$
$\newcommand{\hatcurLBiSthreeeccen}{\ensuremath{0.0530}}$
$\newcommand{\hatcurLBiiSthreeeccen}{\ensuremath{0.1348}}$
$\newcommand{\hatcurLBiSfoureccen}{\ensuremath{0.0565}}$
$\newcommand{\hatcurLBiiSfoureccen}{\ensuremath{0.1029}}$
$\newcommand{\hatcurISOmeccen}{\ensuremath{0.3892\pm0.0045}}$
$\newcommand{\hatcurISOmshorteccen}{\ensuremath{0.39}}$
$\newcommand{\hatcurISOmlongeccen}{\ensuremath{0.3892\pm0.0045}}$
$\newcommand{\hatcurISOreccen}{\ensuremath{0.3720\pm0.0022}}$
$\newcommand{\hatcurISOrshorteccen}{\ensuremath{0.37}}$
$\newcommand{\hatcurISOrlongeccen}{\ensuremath{0.3720\pm0.0022}}$
$\newcommand{\hatcurISOrhoeccen}{\ensuremath{10.64\pm0.23}}$
$\newcommand{\hatcurISOrholongeccen}{\ensuremath{10.64\pm0.23}}$
$\newcommand{\hatcurISOloggeccen}{\ensuremath{4.8866\pm0.0073}}$
$\newcommand{\hatcurISOlumeccen}{\ensuremath{0.01627\pm0.00018}}$
$\newcommand{\hatcurISOlumshorteccen}{\ensuremath{0.02}}$
$\newcommand{\hatcurISOteffeccen}{\ensuremath{3380.6\pm7.5}}$
$\newcommand{\hatcurISOzfeheccen}{\ensuremath{0.245\pm0.016}}$
$\newcommand{\hatcurISOageeccen}{\ensuremath{1.07_{-0.57}^{+2.21}}}$
$\newcommand{\hatcurISOspececcen}{M}$
$\newcommand{\hatcurRVKeccen}{\ensuremath{181.5\pm3.5}}$
$\newcommand{\hatcurRVKtwosiglimeccen}{\ensuremath{<187.2}}$
$\newcommand{\hatcurRVrkeccen}{\ensuremath{-0.058_{-0.040}^{+0.061}}}$
$\newcommand{\hatcurRVrheccen}{\ensuremath{0.104_{-0.072}^{+0.037}}}$
$\newcommand{\hatcurRVkeccen}{\ensuremath{-0.0069\pm0.0070}}$
$\newcommand{\hatcurRVheccen}{\ensuremath{0.0134_{-0.0113}^{+0.0085}}}$
$\newcommand{\hatcurRVtroneeccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurRVtrtwoeccen}{\ensuremath{0\pm0}}$
$\newcommand{\hatcurRVgammaeccen}{\ensuremath{-12182.0\pm2.3}}$
$\newcommand{\hatcurRVjittereccen}{\ensuremath{0.5\pm4.5}}$
$\newcommand{\hatcurRVjittertwosiglimeccen}{\ensuremath{<10.3}}$
$\newcommand{\hatcurRVfitrmseccen}{\ensuremath{.1fym}}$
$\newcommand{\hatcurRVecceneccen}{\ensuremath{0.0170\pm0.0086}}$
$\newcommand{\hatcurRVeccentwosiglimeccen}{\ensuremath{<0.029}}$
$\newcommand{\hatcurRVomegaeccen}{\ensuremath{120\pm44}}$
$\newcommand{\hatcurPPieccen}{\ensuremath{88.110\pm0.044}}$
$\newcommand{\hatcurPPgeccen}{\ensuremath{15.53\pm0.41}}$
$\newcommand{\hatcurPPloggeccen}{\ensuremath{3.191\pm0.011}}$
$\newcommand{\hatcurPPareccen}{\ensuremath{15.59\pm0.11}}$
$\newcommand{\hatcurPPareleccen}{\ensuremath{0.02698\pm0.00010}}$
$\newcommand{\hatcurPPrhoeccen}{\ensuremath{0.759\pm0.024}}$
$\newcommand{\hatcurPPmeccen}{\ensuremath{0.655\pm0.014}}$
$\newcommand{\hatcurPPmtwosiglimeccen}{\ensuremath{<0.68}}$
$\newcommand{\hatcurPPmshorteccen}{\ensuremath{0.66}}$
$\newcommand{\hatcurPPmlongeccen}{\ensuremath{0.655\pm0.014}}$
$\newcommand{\hatcurPPmeeccen}{\ensuremath{208.3\pm4.4}}$
$\newcommand{\hatcurPPmeshorteccen}{\ensuremath{208.3}}$
$\newcommand{\hatcurPPmelongeccen}{\ensuremath{208.3\pm4.4}}$
$\newcommand{\hatcurPPreccen}{\ensuremath{1.0226\pm0.0074}}$
$\newcommand{\hatcurPPrshorteccen}{\ensuremath{1.02}}$
$\newcommand{\hatcurPPrlongeccen}{\ensuremath{1.0226\pm0.0074}}$
$\newcommand{\hatcurPPreeccen}{\ensuremath{11.462\pm0.083}}$
$\newcommand{\hatcurPPreshorteccen}{\ensuremath{11.5}}$
$\newcommand{\hatcurPPrelongeccen}{\ensuremath{11.462\pm0.083}}$
$\newcommand{\hatcurPPmrcorreccen}{\ensuremath{-0.06}}$
$\newcommand{\hatcurPPteffeccen}{\ensuremath{605.4\pm1.6}}$
$\newcommand{\hatcurPPthetaeccen}{\ensuremath{0.0885\pm0.0019}}$
$\newcommand{\hatcurPPfluxperieccen}{\ensuremath{3.149\pm0.080}}$
$\newcommand{\hatcurPPfluxperidimeccen}{\ensuremath{7}}$
$\newcommand{\hatcurPPfluxapeccen}{\ensuremath{2.948\pm0.038}}$
$\newcommand{\hatcurPPfluxapdimeccen}{\ensuremath{7}}$
$\newcommand{\hatcurPPfluxavgeccen}{\ensuremath{3.046\pm0.032}}$
$\newcommand{\hatcurPPfluxavgdimeccen}{\ensuremath{7}}$
$\newcommand{\hatcurPPfluxavglogeccen}{\ensuremath{7.4837\pm0.0046}}$
$\newcommand{\hatcurXsecphaseeccen}{\ensuremath{0.4956\pm0.0044}}$
$\newcommand{\hatcurXsecondaryeccen}{\ensuremath{2459686.101\pm0.012}}$
$\newcommand{\hatcurXsecdureccen}{\ensuremath{0.06290\pm0.00086}}$
$\newcommand{\hatcurXsecingdureccen}{\ensuremath{0.01815\pm0.00043}}$
$\newcommand{\hatcurPPphiconjeccen}{\ensuremath{-0.08\pm0.11}}$
$\newcommand{\hatcurPPperieccen}{\ensuremath{2459685.02\pm0.29}}$
$\newcommand{\hatcurPPaequiveccen}{\ensuremath{0.2114\pm0.0011}}$
$\newcommand{\hatcurPPtcirceccen}{\ensuremath{58.9\pm2.7}}$
$\newcommand{\hatcurPPtinfalleccen}{\ensuremath{13490\pm550}}$
$\newcommand{\hatcurXdisteccen}{\ensuremath{72.500_{-0.100}^{+0.200}}}$
$\newcommand{\hatcurXAveccen}{\ensuremath{0.064\pm0.021}}$
$\newcommand{\hatcurXdistredeccen}{\ensuremath{72.54\pm0.13}}$
$\newcommand{\hatcurXEBVeccen}{\ensuremath{0.0200\pm0.0069}}$
$\newcommand{\hatcurCCpmraeccen}{\ensuremath{-170.503\pm0.028}}$
$\newcommand{\hatcurCCpmdececcen}{\ensuremath{-64.264\pm0.023}}$
$\newcommand{\hatcurCCpmeccen}{\ensuremath{182.212\pm0.036}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\forloop$}[5][1]$
$\newcommand{$\tbd$}[1]{{\par\bf$\textsc${TBD: #1\\}}}$
$\newcommand{$\ctbd$}[1]$
$\newcommand{$\cor$}{\textcolor{red}{(corr?) }}$
$\newcommand{$\spl$}{\textcolor{red}{(spl?) }}$
$\newcommand{$\ii$}{\'\i }$
$\newcommand{$\oo$}{\H{o}}$
$\newcommand{$\uu$}{\H u}$
$\newcommand{$\lc$}{light curve}$
$\newcommand{$\lc$s}{light curves}$
$\newcommand{$\Lc$}{Light curve}$
$\newcommand{$\Lc$s}{Light curves}$
$\newcommand{$\avg$}[1]{$\ensuremath${\langle #1\rangle}}$
$\newcommand{$\med$}[1]{$\ensuremath${\langle #1\rangle_{med}}}$
$\newcommand{$\dpt$}{data-point}$
$\newcommand{$\dpt$s}{data-points}$
$\newcommand{$\tel$}{telescope}$
$\newcommand{$\magn$}{magnitude}$
$\newcommand{$\stan$}{standard}$
$\newcommand{$\aper$}{aperture}$
$\newcommand{$\oo$t}{out-of-transit}$
$\newcommand{$\OOT$}{Out-of-Transit}$
$\newcommand{$\cfa$}{Harvard-Smithsonian Center for Astrophysics (CfA)}$
$\newcommand{$\cfa$digi}{CfA Speedometers}$
$\newcommand{$\cmd$}{color-magnitude diagram}$
$\newcommand{$\conc$}[1]{\noindent\par{\noindent{\mathbf \Longrightarrow \bf #1}}}$
$\newcommand{$\diam$}{$\ensuremath${\oslash}}$
$\newcommand{$\ccdsize$}[1]{$\ensuremath${\rm #1\times\rm#1}}$
$\newcommand{$\fovsize$}[2]{$\ensuremath${\rm #1 #2\times\rm#1 #2}}$
$\newcommand{$\tsize$}[1]{\mbox{\rm #1 m}}$
$\newcommand{$\band$}[1]{$\ensuremath${#1}~band}$
$\newcommand{$\ordo$}{$\ensuremath${\mathcal{O}}}$
$\newcommand{$\chisq$}{$\ensuremath${\chi^2}}$
$\newcommand{$\RA$}[3]{$\ensuremath${#1^{\mathrm h}#2^{\mathrm m}#3^{\mathrm s}}}$
$\newcommand{$\DEC$}[3]{$\ensuremath${#1^{\mathrm d}#2^{\mathrm m}#3^{\mathrm s}}}$
$\newcommand{$\ghr$}{$\ensuremath${^h}}$
$\newcommand{$\gmin$}{$\ensuremath${^m}}$
$\newcommand{$\Ks$}{$\ensuremath${K_s}}$
$\newcommand{$\masy$}{$\ensuremath${\rm mas yr^{-1}}}$
$\newcommand{$\kms$}{$\ensuremath${\rm km s^{-1}}}$
$\newcommand{$\ms$}{$\ensuremath${\rm m s^{-1}}}$
$\newcommand{$\ms$d}{$\ensuremath${\rm m s^{-1} d^{-1}}}$
$\newcommand{$\ms$s}{$\ensuremath${\rm m s^{-2}}}$
$\newcommand{$\gcmc$}{$\ensuremath${\rm g cm^{-3}}}$
$\newcommand{$\ergscmsq$}{$\ensuremath${\rm erg s^{-1} cm^{-2}}}$
$\newcommand{$\C$}{$\ensuremath${^{\circ}C\;}}$
$\newcommand{$\el$}{$\ensuremath${e^-}}$
$\newcommand{$\sqarcsec$}{$\ensuremath${\Box^{\prime\prime}}}$
$\newcommand{$\sqarcdeg$}{$\ensuremath${\Box^{\circ}}}$
$\newcommand{$\pxs$}{$\ensuremath${\rm $\arcsec$ pixel^{-1}}}$
$\newcommand{$\aduel$}{$\ensuremath${\lbrack ADU/$\el$ \rbrack}}$
$\newcommand{$\el$adu}{$\ensuremath${\lbrack $\el$/ADU \rbrack}}$
$\newcommand{$\adupixs$}{$\ensuremath${\rm ADU/(pix  s)}}$
$\newcommand{$\el$pixs}{$\ensuremath${\rm $\el$/(pix  s)}}$
$\newcommand{$\masy$r}{$\ensuremath${\rm mas yr^{-1}}}$
$\newcommand{$\ms$ini}{$\ensuremath${m \sin i}}$
$\newcommand{$\mplsini$}{$\ensuremath${$\mpl$\sin i}}$
$\newcommand{$\teff$}{$\ensuremath${T_{\rm eff}}}$
$\newcommand{$\logg$}{$\ensuremath${\log{g}}}$
$\newcommand{$\vsini$}{$\ensuremath${v \sin{i}}}$
$\newcommand{$\feh$}{$\ensuremath${\rm[Fe/H]}}$
$\newcommand{$\logl$}{$\ensuremath${\log{L}}}$
$\newcommand{$\vmac$}{$\ensuremath${v_{\rm mac}}}$
$\newcommand{$\vmic$}{$\ensuremath${v_{\rm mic}}}$
$\newcommand{$\rhk$}{$\ensuremath${R^{\prime}_{\rm HK}}}$
$\newcommand{$\logrhk$}{$\ensuremath${\log$\rhk$}}$
$\newcommand{$\Savg$}{$\ensuremath${\langle S\rangle}}$
$\newcommand{$\vic$}{$\ensuremath${V\!-\!I_C}}$
$\newcommand{$\ebv$}{$\ensuremath${E(B\!-\!V)}}$
$\newcommand{$\rsun$}{$\ensuremath${R_\sun}}$
$\newcommand{$\ms$un}{$\ensuremath${M_\sun}}$
$\newcommand{$\lsun$}{$\ensuremath${L_\sun}}$
$\newcommand{$\logl$sun}{$\ensuremath${\log{L_\sun}}}$
$\newcommand{$\teff$sun}{$\ensuremath${T_{eff,\sun}}}$
$\newcommand{$\rhosun$}{$\ensuremath${\rho_\sun}}$
$\newcommand{$\logg$sun}{$\ensuremath${\log{g_{\sun}}}}$
$\newcommand{$\rstar$}{$\ensuremath${R_\star}}$
$\newcommand{$\ms$tar}{$\ensuremath${M_\star}}$
$\newcommand{$\lstar$}{$\ensuremath${L_\star}}$
$\newcommand{$\astar$}{$\ensuremath${a_\star}}$
$\newcommand{$\logl$star}{$\ensuremath${\log{L_\star}}}$
$\newcommand{$\teff$star}{$\ensuremath${T_{\rm eff\star}}}$
$\newcommand{$\rhostar$}{$\ensuremath${\rho_\star}}$
$\newcommand{$\logg$star}{$\ensuremath${\log{g_{\star}}}}$
$\newcommand{$\rearth$}{$\ensuremath${R_\earth}}$
$\newcommand{$\mearth$}{$\ensuremath${M_\earth}}$
$\newcommand{$\learth$}{$\ensuremath${L_\earth}}$
$\newcommand{$\teff$earth}{$\ensuremath${T_{\rm eff,\earth}}}$
$\newcommand{$\rhoearth$}{$\ensuremath${\rho_\earth}}$
$\newcommand{$\rpl$}{$\ensuremath${R_{p}}}$
$\newcommand{$\mpl$}{$\ensuremath${M_{p}}}$
$\newcommand{$\lpl$}{$\ensuremath${L_{p}}}$
$\newcommand{$\teff$pl}{$\ensuremath${T_{\rm eff,{p}}}}$
$\newcommand{$\rhopl$}{$\ensuremath${\rho_{p}}}$
$\newcommand{$\ipl$}{$\ensuremath${i_{p}}}$
$\newcommand{$\epl$}{$\ensuremath${e_{p}}}$
$\newcommand{$\gpl$}{$\ensuremath${g_{p}}}$
$\newcommand{$\logg$pl}{$\ensuremath${\log g_{p}}}$
$\newcommand{$\arstar$}{$\ensuremath${a/$\rstar$}}$
$\newcommand{$\zrstar$}{$\ensuremath${\zeta/$\rstar$}}$
$\newcommand{$\rjup$}{$\ensuremath${R_{\rm J}}}$
$\newcommand{$\mjup$}{$\ensuremath${M_{\rm J}}}$
$\newcommand{$\ljup$}{$\ensuremath${L_{\rm J}}}$
$\newcommand{$\teff$jup}{$\ensuremath${T_{eff,{\rm J}}}}$
$\newcommand{$\rhojup$}{$\ensuremath${\rho_{\rm J}}}$
$\newcommand{$\gjup$}{$\ensuremath${\g_{\rm J}}}$
$\newcommand{$\rjup$long}{$\ensuremath${R_{\rm Jup}}}$
$\newcommand{$\mjup$long}{$\ensuremath${M_{\rm Jup}}}$
$\newcommand{$\ljup$long}{$\ensuremath${L_{\rm Jup}}}$
$\newcommand{$\teff$juplong}{$\ensuremath${T_{eff,{\rm Jup}}}}$
$\newcommand{$\rhojup$long}{$\ensuremath${\rho_{\rm Jup}}}$
$\newcommand{$\gjup$long}{$\ensuremath${\g_{\rm Jup}}}$
$\newcommand{$\pack$}[1]{$\textsc${\lowercase{#1}}}$
$\newcommand{$\prog$}[1]{\texttt{\lowercase{#1}}}$
$\newcommand{$\iraf$}{$\pack${iraf}}$
$\newcommand{$\todcor$}{$\prog${todcor}}$
$\newcommand{$\xcsao$}{$\prog${xcsao}}$
$\newcommand{$\daophot$}{$\pack${daophot}}$
$\newcommand{$\fihat$}{$\pack${fihat}}$
$\newcommand{$\fistar$}{$\prog${fistar}}$
$\newcommand{$\fiphot$}{$\prog${fiphot}}$
$\newcommand{$\grmatch$}{$\prog${grmatch}}$
$\newcommand{$\grtrans$}{$\prog${grtrans}}$
$\newcommand{$\refp$}[1]{p.~\pageref{#1}}$
$\newcommand{$\reffig$}[1]{Fig.~\ref{fig:#1}}$
$\newcommand{$\refsec$}[1]{\mbox{\S \ref{sec:#1}}}$
$\newcommand{$\refeq$}[1]{Eq.~\ref{eq:#1}}$
$\newcommand{$\reftab$}[1]{Tab.~\ref{tab:#1}}$
$\newcommand{$\reffig$l}[1]{Figure~\ref{fig:#1}}$
$\newcommand{$\refsec$l}[1]{\mbox{Section \ref{sec:#1}}}$
$\newcommand{$\refeq$l}[1]{Equation~\ref{eq:#1}}$
$\newcommand{$\reftab$l}[1]{Table~\ref{tab:#1}}$
$\newcommand{$\reffig$p}[1]{$\reffig${#1} on \pref{fig:#1}}$
$\newcommand{$\refsec$p}[1]{$\refsec${#1} on \pref{sec:#1}}$
$\newcommand{$\refeq$p}[1]{$\refeq${#1} on \pref{eq:#1}}$
$\newcommand{$\reftab$p}[1]{$\reftab${#1} on \pref{tab:#1}}$
$\newcommand{$\reffig$ls}[2]{Figures~\ref{fig:#1}-\ref{fig:#2}}$
$\newcommand{$\reftab$ls}[2]{Tables~\ref{tab:#1}-\ref{tab:#2}}$
$\newcommand{$\flwof$}{\mbox{FLWO 1.2 m}}$
$\newcommand{$\flwos$}{\mbox{FLWO 1.5 m}}$
$\newcommand{$\flwot$}{\mbox{TopHAT 0.25 m}}$
$\newcommand{$\mmt$}{\mbox{MMT 6.5 m}}$
$\newcommand{$\ssts$}{{\em Spitzer}}$
$\newcommand{$\sstL$}{{\em Spitzer Space Telescope}}$
$\newcommand{$\hst$}{{\em HST}}$
$\newcommand{$\wom$}{\mbox{Wise 1 m}}$
$\newcommand{$\piszkessch$}{Konkoly 0.6 m Schmidt}$
$\newcommand{$\piszkesrcc$}{Konkoly 1 m RCC}$
$\newcommand{$\dscu$}{\mbox{\delta Scuti}}$
$\newcommand{$\gdor$}{\mbox{\gamma Dor}}$
$\newcommand{$\hj$}{hot Jupiter}$
$\newcommand{$\vhj$}{very hot Jupiter}$
$\newcommand{$\hd$}[1]{\mbox{HD #1}}$
$\newcommand{$\BD$}[1]{\mbox{BD #1}}$
$\newcommand{$\hip$}[1]{\mbox{HIP #1}}$
$\newcommand{$\gj$}[1]{\mbox{GJ #1}}$
$\newcommand{$\tbn$}[1]{\tablenotemark{#1}}$
$\newcommand{$\hatcurhtr$}{TOI-3235}$
$\newcommand{$\hatcurfield$}{$\ensuremath${string}}$
$\newcommand{$\hatcurCCra$}{$\ensuremath${13^{\mathrm h}49^{\mathrm m}53.9777{\mathrm s}}}$
$\newcommand{$\hatcurCCdec$}{$\ensuremath${-46{\arcdeg}03{$\arcmin$}58.4541{$\arcsec$}}}$
$\newcommand{$\hatcurCCmag$}{NULL}$
$\newcommand{$\hatcurCCtwomass$}{2MASS~13495398-4603583}$
$\newcommand{$\hatcurCCgsc$}{GSC~NULL}$
$\newcommand{$\hatcurCCgaia$}{GAIA~6107144260247338240}$
$\newcommand{$\hatcurCCgaia$drtwo}{GAIA~DR2~6107144260251920000}$
$\newcommand{$\hatcurCCgaia$drthree}{GAIA~DR3~6107144260251920000}$
$\newcommand{$\hatcurTICID$}{TIC~243641947}$
$\newcommand{$\hatcurCCtassmv$}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmv$short}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmB$}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmB$short}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmI$}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmI$short}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmg$}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmg$short}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmr$}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmr$short}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmi$}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmi$short}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCparallax$}{$\ensuremath${13.781\pm0.027}}$
$\newcommand{$\hatcurCCgaia$mG}{$\ensuremath${14.4605\pm0.0028}}$
$\newcommand{$\hatcurCCgaia$mBP}{$\ensuremath${15.9000\pm0.0042}}$
$\newcommand{$\hatcurCCgaia$mRP}{$\ensuremath${13.2881\pm0.0038}}$
$\newcommand{$\hatcurCCgaia$mGthree}{$\ensuremath${14.4605\pm0.0028}}$
$\newcommand{$\hatcurCCgaia$mBPthree}{$\ensuremath${15.9000\pm0.0042}}$
$\newcommand{$\hatcurCCgaia$mRPthree}{$\ensuremath${13.2881\pm0.0038}}$
$\newcommand{$\hatcurCCtwomass$Jmag}{$\ensuremath${11.706\pm0.025}}$
$\newcommand{$\hatcurCCtwomass$Hmag}{$\ensuremath${11.099\pm0.024}}$
$\newcommand{$\hatcurCCtwomass$Kmag}{$\ensuremath${10.819\pm0.021}}$
$\newcommand{$\hatcurCCcitJmag$}{$\ensuremath${11.695\pm0.026}}$
$\newcommand{$\hatcurCCcitHmag$}{$\ensuremath${11.088\pm0.026}}$
$\newcommand{$\hatcurCCcitKmag$}{$\ensuremath${10.843\pm0.022}}$
$\newcommand{$\hatcurCCbbJmag$}{$\ensuremath${11.787\pm0.029}}$
$\newcommand{$\hatcurCCbbHmag$}{$\ensuremath${11.116\pm0.026}}$
$\newcommand{$\hatcurCCbbKmag$}{$\ensuremath${10.863\pm0.022}}$
$\newcommand{$\hatcurCCesoJmag$}{$\ensuremath${11.796\pm0.033}}$
$\newcommand{$\hatcurCCesoHmag$}{$\ensuremath${11.118\pm0.047}}$
$\newcommand{$\hatcurCCesoKmag$}{$\ensuremath${10.859\pm0.024}}$
$\newcommand{$\hatcurCCesoJHmag$}{$\ensuremath${0.678\pm0.054}}$
$\newcommand{$\hatcurCCesoJKmag$}{$\ensuremath${0.936\pm0.038}}$
$\newcommand{$\hatcurCCesoHKmag$}{$\ensuremath${0.258\pm0.051}}$
$\newcommand{$\hatcurCCWonemag$}{$\ensuremath${10.694\pm0.023}}$
$\newcommand{$\hatcurCCWtwomag$}{$\ensuremath${10.548\pm0.021}}$
$\newcommand{$\hatcurCCWthreemag$}{$\ensuremath${10.458\pm0.064}}$
$\newcommand{$\hatcurCCWfourmag$}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurLCdip$}{$\ensuremath${0.1}}$
$\newcommand{$\hatcurLCrprstar$}{$\ensuremath${0.2828\pm0.0016}}$
$\newcommand{$\hatcurLCbsq$}{$\ensuremath${0.261_{-0.012}^{+0.012}}}$
$\newcommand{$\hatcurLCimp$}{$\ensuremath${0.511_{-0.012}^{+0.011}}}$
$\newcommand{$\hatcurLCzeta$}{$\ensuremath${44.41\pm0.32}}$
$\newcommand{$\hatcurLCdur$}{$\ensuremath${0.06165\pm0.00021}}$
$\newcommand{$\hatcurLCdur$short}{$\ensuremath${0.0616}}$
$\newcommand{$\hatcurLCdur$hr}{$\ensuremath${1.4795\pm0.0049}}$
$\newcommand{$\hatcurLCdur$hrshort}{$\ensuremath${1.480}}$
$\newcommand{$\hatcurLCq$}{$\ensuremath${0.023800\pm0.000086}}$
$\newcommand{$\hatcurLCq$short}{$\ensuremath${0.024}}$
$\newcommand{$\hatcurLCingdur$}{$\ensuremath${0.01765\pm0.00030}}$
$\newcommand{$\hatcurLCP$}{$\ensuremath${2.59261842\pm0.00000041}}$
$\newcommand{$\hatcurLCP$prec}{$\ensuremath${2.5926184}}$
$\newcommand{$\hatcurLCP$short}{$\ensuremath${2.5926}}$
$\newcommand{$\hatcurLCT$}{$\ensuremath${2459690.001730\pm0.000045}}$
$\newcommand{$\hatcurLCT$A}{$\ensuremath${2457797.39028\pm0.00028}}$
$\newcommand{$\hatcurLCT$B}{$\ensuremath${2459762.595048\pm0.000053}}$
$\newcommand{$\hatcurLChatnetmA$}{$\ensuremath${13.62811\pm0.00072}}$
$\newcommand{$\hatcurLCiblendA$}{$\ensuremath${0.982\pm0.022}}$
$\newcommand{$\hatcurLChatnetmB$}{$\ensuremath${13.63138\pm0.00053}}$
$\newcommand{$\hatcurLCiblendB$}{$\ensuremath${0.957\pm0.027}}$
$\newcommand{$\hatcurLChatnetmC$}{$\ensuremath${-7.03640\pm0.00011}}$
$\newcommand{$\hatcurLCiblendC$}{$\ensuremath${1.084\pm0.011}}$
$\newcommand{$\hatcurLChatnetmD$}{$\ensuremath${-6.996030\pm0.000084}}$
$\newcommand{$\hatcurLCiblendD$}{$\ensuremath${1.1248\pm0.0088}}$
$\newcommand{$\hatcurLCrho$}{$\ensuremath${10.99\pm0.13}}$
$\newcommand{$\hatcurSMEiteff$}{$\ensuremath${3196\pm67}}$
$\newcommand{$\hatcurSMEizfeh$}{$\ensuremath${-0.02\pm0.10}}$
$\newcommand{$\hatcurSMEizfeh$short}{$\ensuremath${-0.02}}$
$\newcommand{$\hatcurSMEilogg$}{$\ensuremath${4.80\pm0.50}}$
$\newcommand{$\hatcurSMEivsin$}{$\ensuremath${0\pm50}}$
$\newcommand{$\hatcurSMEivmac$}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurSMEivmic$}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurextraerrMJ$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMJ$twosiglim}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMH$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMH$twosiglim}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMKs$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMKs$twosiglim}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMG$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMG$twosiglim}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMRP$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMRP$twosiglim}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMWone$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMWone$twosiglim}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMWtwo$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMWtwo$twosiglim}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurLBiB$}{$\ensuremath${0.5035}}$
$\newcommand{$\hatcurLBiiB$}{$\ensuremath${0.3376}}$
$\newcommand{$\hatcurLBiV$}{$\ensuremath${0.4840}}$
$\newcommand{$\hatcurLBiiV$}{$\ensuremath${0.3559}}$
$\newcommand{$\hatcurLBiR$}{$\ensuremath${0.4142}}$
$\newcommand{$\hatcurLBiiR$}{$\ensuremath${0.3369}}$
$\newcommand{$\hatcurLBiI$}{$\ensuremath${0.2042}}$
$\newcommand{$\hatcurLBiiI$}{$\ensuremath${0.3912}}$
$\newcommand{$\hatcurLBiu$}{$\ensuremath${0.4328}}$
$\newcommand{$\hatcurLBiiu$}{$\ensuremath${0.3416}}$
$\newcommand{$\hatcurLBig$}{$\ensuremath${0.31\pm0.12}}$
$\newcommand{$\hatcurLBiig$}{$\ensuremath${0.35\pm0.14}}$
$\newcommand{$\hatcurLBir$}{$\ensuremath${0.41\pm0.15}}$
$\newcommand{$\hatcurLBiir$}{$\ensuremath${0.32\pm0.16}}$
$\newcommand{$\hatcurLBii$}{$\ensuremath${0.30\pm0.11}}$
$\newcommand{$\hatcurLBii$i}{$\ensuremath${0.26_{-0.15}^{+0.11}}}$
$\newcommand{$\hatcurLBiz$}{$\ensuremath${0.140_{-0.081}^{+0.106}}}$
$\newcommand{$\hatcurLBii$z}{$\ensuremath${0.17\pm0.13}}$
$\newcommand{$\hatcurLBiJ$}{$\ensuremath${0.126\pm0.072}}$
$\newcommand{$\hatcurLBii$J}{$\ensuremath${0.12\pm0.12}}$
$\newcommand{$\hatcurLBiH$}{$\ensuremath${0.0889}}$
$\newcommand{$\hatcurLBii$H}{$\ensuremath${0.2815}}$
$\newcommand{$\hatcurLBiK$}{$\ensuremath${0.0827}}$
$\newcommand{$\hatcurLBii$K}{$\ensuremath${0.2458}}$
$\newcommand{$\hatcurLBiT$}{$\ensuremath${0.24\pm0.12}}$
$\newcommand{$\hatcurLBii$T}{$\ensuremath${0.33\pm0.16}}$
$\newcommand{$\hatcurLBikep$}{$\ensuremath${0.2909}}$
$\newcommand{$\hatcurLBii$kep}{$\ensuremath${0.4272}}$
$\newcommand{$\hatcurLBiC$}{$\ensuremath${0.2393}}$
$\newcommand{$\hatcurLBii$C}{$\ensuremath${0.4323}}$
$\newcommand{$\hatcurLBiM$}{$\ensuremath${0.4020}}$
$\newcommand{$\hatcurLBii$M}{$\ensuremath${0.4034}}$
$\newcommand{$\hatcurLBiSone$}{$\ensuremath${0.0655}}$
$\newcommand{$\hatcurLBii$Sone}{$\ensuremath${0.1827}}$
$\newcommand{$\hatcurLBiStwo$}{$\ensuremath${0.0480}}$
$\newcommand{$\hatcurLBii$Stwo}{$\ensuremath${0.1526}}$
$\newcommand{$\hatcurLBiSthree$}{$\ensuremath${0.0530}}$
$\newcommand{$\hatcurLBii$Sthree}{$\ensuremath${0.1348}}$
$\newcommand{$\hatcurLBiSfour$}{$\ensuremath${0.0565}}$
$\newcommand{$\hatcurLBii$Sfour}{$\ensuremath${0.1029}}$
$\newcommand{$\hatcurISOm$}{$\ensuremath${0.3939\pm0.0030}}$
$\newcommand{$\hatcurISOm$short}{$\ensuremath${0.39}}$
$\newcommand{$\hatcurISOm$long}{$\ensuremath${0.3939\pm0.0030}}$
$\newcommand{$\hatcurISOr$}{$\ensuremath${0.3697\pm0.0018}}$
$\newcommand{$\hatcurISOr$short}{$\ensuremath${0.37}}$
$\newcommand{$\hatcurISOr$long}{$\ensuremath${0.3697\pm0.0018}}$
$\newcommand{$\hatcurISOr$ho}{$\ensuremath${10.99\pm0.13}}$
$\newcommand{$\hatcurISOr$holong}{$\ensuremath${10.99\pm0.13}}$
$\newcommand{$\hatcurISOlogg$}{$\ensuremath${4.8976\pm0.0035}}$
$\newcommand{$\hatcurISOlum$}{$\ensuremath${0.01623\pm0.00018}}$
$\newcommand{$\hatcurISOlum$short}{$\ensuremath${0.02}}$
$\newcommand{$\hatcurISOteff$}{$\ensuremath${3388.8\pm5.9}}$
$\newcommand{$\hatcurISOteff$short}{$\ensuremath${3389}}$
$\newcommand{$\hatcurISOzfeh$}{$\ensuremath${0.264_{-0.017}^{+0.013}}}$
$\newcommand{$\hatcurISOage$}{$\ensuremath${0.394_{-0.090}^{+0.152}}}$
$\newcommand{$\hatcurISOspec$}{M}$
$\newcommand{$\hatcurRVK$}{$\ensuremath${182.9\pm3.3}}$
$\newcommand{$\hatcurRVK$twosiglim}{$\ensuremath${<188.1}}$
$\newcommand{$\hatcurRVrk$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurRVrh$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurRVk$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurRVh$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurRVtrone$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurRVtrtwo$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurRVgamma$}{$\ensuremath${-12182.3\pm2.4}}$
$\newcommand{$\hatcurRVjitter$}{$\ensuremath${0.1\pm4.2}}$
$\newcommand{$\hatcurRVjitter$twosiglim}{$\ensuremath${<9.6}}$
$\newcommand{$\hatcurRVfitrms$}{$\ensuremath${.1fym}}$
$\newcommand{$\hatcurRVeccen$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurRVeccen$twosiglim}{$\ensuremath${<0.000}}$
$\newcommand{$\hatcurRVomega$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurPPi$}{$\ensuremath${88.140\pm0.046}}$
$\newcommand{$\hatcurPPg$}{$\ensuremath${15.90\pm0.37}}$
$\newcommand{$\hatcurPPloggold$}{$\ensuremath${3.202\pm0.010}}$
$\newcommand{$\hatcurPPlogg$}{$\ensuremath${3.202\pm0.041}}$
$\newcommand{$\hatcurPParold$}{$\ensuremath${15.754\pm0.062}}$
$\newcommand{$\hatcurPPar$}{$\ensuremath${15.75\pm0.73}}$
$\newcommand{$\hatcurPPar$elold}{$\ensuremath${0.027089\pm0.000069}}$
$\newcommand{$\hatcurPPar$el}{$\ensuremath${0.02709\pm0.00046}}$
$\newcommand{$\hatcurPPrhoold$}{$\ensuremath${0.783\pm0.022}}$
$\newcommand{$\hatcurPPrho$}{$\ensuremath${0.78\pm0.11}}$
$\newcommand{$\hatcurPPmold$}{$\ensuremath${0.665\pm0.013}}$
$\newcommand{$\hatcurPPm$}{$\ensuremath${0.665\pm0.025}}$
$\newcommand{$\hatcurPPm$twosiglim}{$\ensuremath${<0.68}}$
$\newcommand{$\hatcurPPm$short}{$\ensuremath${0.66}}$
$\newcommand{$\hatcurPPm$long}{$\ensuremath${0.665\pm0.025}}$
$\newcommand{$\hatcurPPm$e}{$\ensuremath${211.3\pm4.0}}$
$\newcommand{$\hatcurPPm$eshort}{$\ensuremath${211.3}}$
$\newcommand{$\hatcurPPm$elong}{$\ensuremath${211.3\pm4.0}}$
$\newcommand{$\hatcurPProld$}{$\ensuremath${1.0174\pm0.0078}}$
$\newcommand{$\hatcurPPr$}{$\ensuremath${1.017\pm0.044}}$
$\newcommand{$\hatcurPPr$short}{$\ensuremath${1.02}}$
$\newcommand{$\hatcurPPr$long}{$\ensuremath${1.017\pm0.044}}$
$\newcommand{$\hatcurPPr$e}{$\ensuremath${11.404\pm0.087}}$
$\newcommand{$\hatcurPPr$eshort}{$\ensuremath${11.4}}$
$\newcommand{$\hatcurPPr$elong}{$\ensuremath${11.404\pm0.087}}$
$\newcommand{$\hatcurPPm$rcorr}{$\ensuremath${0.09}}$
$\newcommand{$\hatcurPPteffold$}{$\ensuremath${603.8\pm1.2}}$
$\newcommand{$\hatcurPPteff$}{$\ensuremath${604\pm19}}$
$\newcommand{$\hatcurPPteff$short}{$\ensuremath${604}}$
$\newcommand{$\hatcurPPthetaold$}{$\ensuremath${0.0896\pm0.0018}}$
$\newcommand{$\hatcurPPtheta$}{$\ensuremath${0.0896\pm0.0042}}$
$\newcommand{$\hatcurPPfluxperi$}{$\ensuremath${3.014\pm0.024}}$
$\newcommand{$\hatcurPPfluxperi$dim}{$\ensuremath${7}}$
$\newcommand{$\hatcurPPfluxap$}{$\ensuremath${3.014\pm0.024}}$
$\newcommand{$\hatcurPPfluxap$dim}{$\ensuremath${7}}$
$\newcommand{$\hatcurPPfluxavg$}{$\ensuremath${3.014\pm0.024}}$
$\newcommand{$\hatcurPPfluxavg$dim}{$\ensuremath${7}}$
$\newcommand{$\hatcurPPfluxavg$logold}{$\ensuremath${7.4791\pm0.0035}}$
$\newcommand{$\hatcurPPfluxavg$log}{$\ensuremath${7.479\pm0.018}}$
$\newcommand{$\hatcurXsecphase$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurXsecondary$}{$\ensuremath${2459691.298040\pm0.000045}}$
$\newcommand{$\hatcurXsecdur$}{$\ensuremath${0.06165\pm0.00021}}$
$\newcommand{$\hatcurXsecingdur$}{$\ensuremath${0.01765\pm0.00030}}$
$\newcommand{$\hatcurPPphiconj$}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurPPperi$}{$\ensuremath${2459689.353577\pm0.000045}}$
$\newcommand{$\hatcurPPaequiv$}{$\ensuremath${0.21250\pm0.00085}}$
$\newcommand{$\hatcurPPtcirc$}{$\ensuremath${62.0\pm2.5}}$
$\newcommand{$\hatcurPPtinfall$}{$\ensuremath${14150\pm370}}$
$\newcommand{$\hatcurXdist$}{$\ensuremath${72.50\pm0.13}}$
$\newcommand{$\hatcurXAv$}{$\ensuremath${0.064\pm0.021}}$
$\newcommand{$\hatcurXdist$red}{$\ensuremath${72.50\pm0.12}}$
$\newcommand{$\hatcurXEBV$}{$\ensuremath${0.0210_{-0.0070}^{+0.0050}}}$
$\newcommand{$\hatcurCCpmra$}{$\ensuremath${-170.503\pm0.028}}$
$\newcommand{$\hatcurCCpmdec$}{$\ensuremath${-64.264\pm0.023}}$
$\newcommand{$\hatcurCCpm$}{$\ensuremath${182.212\pm0.036}}$
$\newcommand{$\hatcurhtr$eccen}{TOI3235}$
$\newcommand{$\hatcurfield$eccen}{$\ensuremath${string}}$
$\newcommand{$\hatcurCCra$eccen}{$\ensuremath${13^{\mathrm h}49^{\mathrm m}53.9777{\mathrm s}}}$
$\newcommand{$\hatcurCCdec$eccen}{$\ensuremath${-46{\arcdeg}03{$\arcmin$}58.4541{$\arcsec$}}}$
$\newcommand{$\hatcurCCmag$eccen}{NULL}$
$\newcommand{$\hatcurCCtwomass$eccen}{2MASS~13495398-4603583}$
$\newcommand{$\hatcurCCgsc$eccen}{GSC~NULL}$
$\newcommand{$\hatcurCCgaia$eccen}{GAIA~6107144260247338240}$
$\newcommand{$\hatcurCCgaia$drtwoeccen}{GAIA~DR2~6107144260251920000}$
$\newcommand{$\hatcurCCtassmv$eccen}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmv$shorteccen}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmB$eccen}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmB$shorteccen}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmI$eccen}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmI$shorteccen}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmg$eccen}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmg$shorteccen}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmr$eccen}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmr$shorteccen}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCtassmi$eccen}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurCCtassmi$shorteccen}{$\ensuremath${0.0}}$
$\newcommand{$\hatcurCCparallax$eccen}{$\ensuremath${13.781\pm0.027}}$
$\newcommand{$\hatcurCCgaia$mGeccen}{$\ensuremath${14.4605\pm0.0028}}$
$\newcommand{$\hatcurCCgaia$mBPeccen}{$\ensuremath${15.9000\pm0.0042}}$
$\newcommand{$\hatcurCCgaia$mRPeccen}{$\ensuremath${13.2881\pm0.0038}}$
$\newcommand{$\hatcurCCtwomass$Jmageccen}{$\ensuremath${11.706\pm0.025}}$
$\newcommand{$\hatcurCCtwomass$Hmageccen}{$\ensuremath${11.099\pm0.024}}$
$\newcommand{$\hatcurCCtwomass$Kmageccen}{$\ensuremath${10.819\pm0.021}}$
$\newcommand{$\hatcurCCcitJmag$eccen}{$\ensuremath${11.695\pm0.026}}$
$\newcommand{$\hatcurCCcitHmag$eccen}{$\ensuremath${11.088\pm0.026}}$
$\newcommand{$\hatcurCCcitKmag$eccen}{$\ensuremath${10.843\pm0.022}}$
$\newcommand{$\hatcurCCbbJmag$eccen}{$\ensuremath${11.787\pm0.029}}$
$\newcommand{$\hatcurCCbbHmag$eccen}{$\ensuremath${11.116\pm0.026}}$
$\newcommand{$\hatcurCCbbKmag$eccen}{$\ensuremath${10.863\pm0.022}}$
$\newcommand{$\hatcurCCesoJmag$eccen}{$\ensuremath${11.796\pm0.033}}$
$\newcommand{$\hatcurCCesoHmag$eccen}{$\ensuremath${11.118\pm0.047}}$
$\newcommand{$\hatcurCCesoKmag$eccen}{$\ensuremath${10.859\pm0.024}}$
$\newcommand{$\hatcurCCesoJHmag$eccen}{$\ensuremath${0.678\pm0.054}}$
$\newcommand{$\hatcurCCesoJKmag$eccen}{$\ensuremath${0.936\pm0.038}}$
$\newcommand{$\hatcurCCesoHKmag$eccen}{$\ensuremath${0.258\pm0.051}}$
$\newcommand{$\hatcurCCWonemag$eccen}{$\ensuremath${10.694\pm0.023}}$
$\newcommand{$\hatcurCCWtwomag$eccen}{$\ensuremath${10.548\pm0.021}}$
$\newcommand{$\hatcurCCWthreemag$eccen}{$\ensuremath${10.458\pm0.064}}$
$\newcommand{$\hatcurCCWfourmag$eccen}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurLCdip$eccen}{$\ensuremath${0.1}}$
$\newcommand{$\hatcurLCrprstar$eccen}{$\ensuremath${0.2826\pm0.0015}}$
$\newcommand{$\hatcurLCbsq$eccen}{$\ensuremath${0.258_{-0.013}^{+0.012}}}$
$\newcommand{$\hatcurLCimp$eccen}{$\ensuremath${0.507_{-0.013}^{+0.012}}}$
$\newcommand{$\hatcurLCzeta$eccen}{$\ensuremath${44.43\pm0.30}}$
$\newcommand{$\hatcurLCdur$eccen}{$\ensuremath${0.06154\pm0.00021}}$
$\newcommand{$\hatcurLCdur$shorteccen}{$\ensuremath${0.0615}}$
$\newcommand{$\hatcurLCdur$hreccen}{$\ensuremath${1.4770\pm0.0051}}$
$\newcommand{$\hatcurLCdur$hrshorteccen}{$\ensuremath${1.477}}$
$\newcommand{$\hatcurLCq$eccen}{$\ensuremath${0.023700\pm0.000096}}$
$\newcommand{$\hatcurLCq$shorteccen}{$\ensuremath${0.024}}$
$\newcommand{$\hatcurLCingdur$eccen}{$\ensuremath${0.01753\pm0.00031}}$
$\newcommand{$\hatcurLCP$eccen}{$\ensuremath${2.59261840\pm0.00000040}}$
$\newcommand{$\hatcurLCP$prececcen}{$\ensuremath${2.5926184}}$
$\newcommand{$\hatcurLCP$shorteccen}{$\ensuremath${2.5926}}$
$\newcommand{$\hatcurLCT$eccen}{$\ensuremath${2459684.816480\pm0.000046}}$
$\newcommand{$\hatcurLCT$Aeccen}{$\ensuremath${2457797.39029\pm0.00026}}$
$\newcommand{$\hatcurLCT$Beccen}{$\ensuremath${2459762.595037\pm0.000053}}$
$\newcommand{$\hatcurLChatnetmA$eccen}{$\ensuremath${13.62805\pm0.00074}}$
$\newcommand{$\hatcurLCiblendA$eccen}{$\ensuremath${0.981\pm0.022}}$
$\newcommand{$\hatcurLChatnetmB$eccen}{$\ensuremath${13.63141\pm0.00051}}$
$\newcommand{$\hatcurLCiblendB$eccen}{$\ensuremath${0.965\pm0.029}}$
$\newcommand{$\hatcurLChatnetmC$eccen}{$\ensuremath${-7.03641\pm0.00011}}$
$\newcommand{$\hatcurLCiblendC$eccen}{$\ensuremath${1.085\pm0.011}}$
$\newcommand{$\hatcurLChatnetmD$eccen}{$\ensuremath${-6.996010\pm0.000083}}$
$\newcommand{$\hatcurLCiblendD$eccen}{$\ensuremath${1.1257\pm0.0089}}$
$\newcommand{$\hatcurLCrho$eccen}{$\ensuremath${10.64\pm0.23}}$
$\newcommand{$\hatcurSMEiteff$eccen}{$\ensuremath${3196\pm67}}$
$\newcommand{$\hatcurSMEizfeh$eccen}{$\ensuremath${-0.02\pm0.10}}$
$\newcommand{$\hatcurSMEizfeh$shorteccen}{$\ensuremath${-0.02}}$
$\newcommand{$\hatcurSMEilogg$eccen}{$\ensuremath${4.80\pm0.50}}$
$\newcommand{$\hatcurSMEivsin$eccen}{$\ensuremath${0\pm50}}$
$\newcommand{$\hatcurSMEivmac$eccen}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurSMEivmic$eccen}{$\ensuremath${nff\pmnff}}$
$\newcommand{$\hatcurextraerrMJ$eccen}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMJ$twosiglimeccen}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMH$eccen}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMH$twosiglimeccen}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMKs$eccen}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMKs$twosiglimeccen}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMG$eccen}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMG$twosiglimeccen}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMRP$eccen}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMRP$twosiglimeccen}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMWone$eccen}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMWone$twosiglimeccen}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurextraerrMWtwo$eccen}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurextraerrMWtwo$twosiglimeccen}{$\ensuremath${<0.0200}}$
$\newcommand{$\hatcurLBiB$eccen}{$\ensuremath${0.5035}}$
$\newcommand{$\hatcurLBiiB$eccen}{$\ensuremath${0.3376}}$
$\newcommand{$\hatcurLBiV$eccen}{$\ensuremath${0.4840}}$
$\newcommand{$\hatcurLBiiV$eccen}{$\ensuremath${0.3559}}$
$\newcommand{$\hatcurLBiR$eccen}{$\ensuremath${0.4142}}$
$\newcommand{$\hatcurLBiiR$eccen}{$\ensuremath${0.3369}}$
$\newcommand{$\hatcurLBiI$eccen}{$\ensuremath${0.2042}}$
$\newcommand{$\hatcurLBiiI$eccen}{$\ensuremath${0.3912}}$
$\newcommand{$\hatcurLBiu$eccen}{$\ensuremath${0.4328}}$
$\newcommand{$\hatcurLBiiu$eccen}{$\ensuremath${0.3416}}$
$\newcommand{$\hatcurLBig$eccen}{$\ensuremath${0.34\pm0.13}}$
$\newcommand{$\hatcurLBiig$eccen}{$\ensuremath${0.31\pm0.16}}$
$\newcommand{$\hatcurLBir$eccen}{$\ensuremath${0.39\pm0.15}}$
$\newcommand{$\hatcurLBiir$eccen}{$\ensuremath${0.33\pm0.17}}$
$\newcommand{$\hatcurLBii$eccen}{$\ensuremath${0.303\pm0.097}}$
$\newcommand{$\hatcurLBii$ieccen}{$\ensuremath${0.24\pm0.13}}$
$\newcommand{$\hatcurLBiz$eccen}{$\ensuremath${0.154\pm0.096}}$
$\newcommand{$\hatcurLBii$zeccen}{$\ensuremath${0.16\pm0.14}}$
$\newcommand{$\hatcurLBiJ$eccen}{$\ensuremath${0.128\pm0.082}}$
$\newcommand{$\hatcurLBii$Jeccen}{$\ensuremath${0.12\pm0.13}}$
$\newcommand{$\hatcurLBiH$eccen}{$\ensuremath${0.0889}}$
$\newcommand{$\hatcurLBii$Heccen}{$\ensuremath${0.2815}}$
$\newcommand{$\hatcurLBiK$eccen}{$\ensuremath${0.0827}}$
$\newcommand{$\hatcurLBii$Keccen}{$\ensuremath${0.2458}}$
$\newcommand{$\hatcurLBiT$eccen}{$\ensuremath${0.24\pm0.11}}$
$\newcommand{$\hatcurLBii$Teccen}{$\ensuremath${0.32\pm0.15}}$
$\newcommand{$\hatcurLBikep$eccen}{$\ensuremath${0.2909}}$
$\newcommand{$\hatcurLBii$kepeccen}{$\ensuremath${0.4272}}$
$\newcommand{$\hatcurLBiC$eccen}{$\ensuremath${0.2393}}$
$\newcommand{$\hatcurLBii$Ceccen}{$\ensuremath${0.4323}}$
$\newcommand{$\hatcurLBiM$eccen}{$\ensuremath${0.4020}}$
$\newcommand{$\hatcurLBii$Meccen}{$\ensuremath${0.4034}}$
$\newcommand{$\hatcurLBiSone$eccen}{$\ensuremath${0.0655}}$
$\newcommand{$\hatcurLBii$Soneeccen}{$\ensuremath${0.1827}}$
$\newcommand{$\hatcurLBiStwo$eccen}{$\ensuremath${0.0480}}$
$\newcommand{$\hatcurLBii$Stwoeccen}{$\ensuremath${0.1526}}$
$\newcommand{$\hatcurLBiSthree$eccen}{$\ensuremath${0.0530}}$
$\newcommand{$\hatcurLBii$Sthreeeccen}{$\ensuremath${0.1348}}$
$\newcommand{$\hatcurLBiSfour$eccen}{$\ensuremath${0.0565}}$
$\newcommand{$\hatcurLBii$Sfoureccen}{$\ensuremath${0.1029}}$
$\newcommand{$\hatcurISOm$eccen}{$\ensuremath${0.3892\pm0.0045}}$
$\newcommand{$\hatcurISOm$shorteccen}{$\ensuremath${0.39}}$
$\newcommand{$\hatcurISOm$longeccen}{$\ensuremath${0.3892\pm0.0045}}$
$\newcommand{$\hatcurISOr$eccen}{$\ensuremath${0.3720\pm0.0022}}$
$\newcommand{$\hatcurISOr$shorteccen}{$\ensuremath${0.37}}$
$\newcommand{$\hatcurISOr$longeccen}{$\ensuremath${0.3720\pm0.0022}}$
$\newcommand{$\hatcurISOr$hoeccen}{$\ensuremath${10.64\pm0.23}}$
$\newcommand{$\hatcurISOr$holongeccen}{$\ensuremath${10.64\pm0.23}}$
$\newcommand{$\hatcurISOlogg$eccen}{$\ensuremath${4.8866\pm0.0073}}$
$\newcommand{$\hatcurISOlum$eccen}{$\ensuremath${0.01627\pm0.00018}}$
$\newcommand{$\hatcurISOlum$shorteccen}{$\ensuremath${0.02}}$
$\newcommand{$\hatcurISOteff$eccen}{$\ensuremath${3380.6\pm7.5}}$
$\newcommand{$\hatcurISOzfeh$eccen}{$\ensuremath${0.245\pm0.016}}$
$\newcommand{$\hatcurISOage$eccen}{$\ensuremath${1.07_{-0.57}^{+2.21}}}$
$\newcommand{$\hatcurISOspec$eccen}{M}$
$\newcommand{$\hatcurRVK$eccen}{$\ensuremath${181.5\pm3.5}}$
$\newcommand{$\hatcurRVK$twosiglimeccen}{$\ensuremath${<187.2}}$
$\newcommand{$\hatcurRVrk$eccen}{$\ensuremath${-0.058_{-0.040}^{+0.061}}}$
$\newcommand{$\hatcurRVrh$eccen}{$\ensuremath${0.104_{-0.072}^{+0.037}}}$
$\newcommand{$\hatcurRVk$eccen}{$\ensuremath${-0.0069\pm0.0070}}$
$\newcommand{$\hatcurRVh$eccen}{$\ensuremath${0.0134_{-0.0113}^{+0.0085}}}$
$\newcommand{$\hatcurRVtrone$eccen}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurRVtrtwo$eccen}{$\ensuremath${0\pm0}}$
$\newcommand{$\hatcurRVgamma$eccen}{$\ensuremath${-12182.0\pm2.3}}$
$\newcommand{$\hatcurRVjitter$eccen}{$\ensuremath${0.5\pm4.5}}$
$\newcommand{$\hatcurRVjitter$twosiglimeccen}{$\ensuremath${<10.3}}$
$\newcommand{$\hatcurRVfitrms$eccen}{$\ensuremath${.1fym}}$
$\newcommand{$\hatcurRVeccen$eccen}{$\ensuremath${0.0170\pm0.0086}}$
$\newcommand{$\hatcurRVeccen$twosiglimeccen}{$\ensuremath${<0.029}}$
$\newcommand{$\hatcurRVomega$eccen}{$\ensuremath${120\pm44}}$
$\newcommand{$\hatcurPPi$eccen}{$\ensuremath${88.110\pm0.044}}$
$\newcommand{$\hatcurPPg$eccen}{$\ensuremath${15.53\pm0.41}}$
$\newcommand{$\hatcurPPlogg$eccen}{$\ensuremath${3.191\pm0.011}}$
$\newcommand{$\hatcurPPar$eccen}{$\ensuremath${15.59\pm0.11}}$
$\newcommand{$\hatcurPPar$eleccen}{$\ensuremath${0.02698\pm0.00010}}$
$\newcommand{$\hatcurPPrho$eccen}{$\ensuremath${0.759\pm0.024}}$
$\newcommand{$\hatcurPPm$eccen}{$\ensuremath${0.655\pm0.014}}$
$\newcommand{$\hatcurPPm$twosiglimeccen}{$\ensuremath${<0.68}}$
$\newcommand{$\hatcurPPm$shorteccen}{$\ensuremath${0.66}}$
$\newcommand{$\hatcurPPm$longeccen}{$\ensuremath${0.655\pm0.014}}$
$\newcommand{$\hatcurPPm$eeccen}{$\ensuremath${208.3\pm4.4}}$
$\newcommand{$\hatcurPPm$eshorteccen}{$\ensuremath${208.3}}$
$\newcommand{$\hatcurPPm$elongeccen}{$\ensuremath${208.3\pm4.4}}$
$\newcommand{$\hatcurPPr$eccen}{$\ensuremath${1.0226\pm0.0074}}$
$\newcommand{$\hatcurPPr$shorteccen}{$\ensuremath${1.02}}$
$\newcommand{$\hatcurPPr$longeccen}{$\ensuremath${1.0226\pm0.0074}}$
$\newcommand{$\hatcurPPr$eeccen}{$\ensuremath${11.462\pm0.083}}$
$\newcommand{$\hatcurPPr$eshorteccen}{$\ensuremath${11.5}}$
$\newcommand{$\hatcurPPr$elongeccen}{$\ensuremath${11.462\pm0.083}}$
$\newcommand{$\hatcurPPm$rcorreccen}{$\ensuremath${-0.06}}$
$\newcommand{$\hatcurPPteff$eccen}{$\ensuremath${605.4\pm1.6}}$
$\newcommand{$\hatcurPPtheta$eccen}{$\ensuremath${0.0885\pm0.0019}}$
$\newcommand{$\hatcurPPfluxperi$eccen}{$\ensuremath${3.149\pm0.080}}$
$\newcommand{$\hatcurPPfluxperi$dimeccen}{$\ensuremath${7}}$
$\newcommand{$\hatcurPPfluxap$eccen}{$\ensuremath${2.948\pm0.038}}$
$\newcommand{$\hatcurPPfluxap$dimeccen}{$\ensuremath${7}}$
$\newcommand{$\hatcurPPfluxavg$eccen}{$\ensuremath${3.046\pm0.032}}$
$\newcommand{$\hatcurPPfluxavg$dimeccen}{$\ensuremath${7}}$
$\newcommand{$\hatcurPPfluxavg$logeccen}{$\ensuremath${7.4837\pm0.0046}}$
$\newcommand{$\hatcurXsecphase$eccen}{$\ensuremath${0.4956\pm0.0044}}$
$\newcommand{$\hatcurXsecondary$eccen}{$\ensuremath${2459686.101\pm0.012}}$
$\newcommand{$\hatcurXsecdur$eccen}{$\ensuremath${0.06290\pm0.00086}}$
$\newcommand{$\hatcurXsecingdur$eccen}{$\ensuremath${0.01815\pm0.00043}}$
$\newcommand{$\hatcurPPphiconj$eccen}{$\ensuremath${-0.08\pm0.11}}$
$\newcommand{$\hatcurPPperi$eccen}{$\ensuremath${2459685.02\pm0.29}}$
$\newcommand{$\hatcurPPaequiv$eccen}{$\ensuremath${0.2114\pm0.0011}}$
$\newcommand{$\hatcurPPtcirc$eccen}{$\ensuremath${58.9\pm2.7}}$
$\newcommand{$\hatcurPPtinfall$eccen}{$\ensuremath${13490\pm550}}$
$\newcommand{$\hatcurXdist$eccen}{$\ensuremath${72.500_{-0.100}^{+0.200}}}$
$\newcommand{$\hatcurXAv$eccen}{$\ensuremath${0.064\pm0.021}}$
$\newcommand{$\hatcurXdist$redeccen}{$\ensuremath${72.54\pm0.13}}$
$\newcommand{$\hatcurXEBV$eccen}{$\ensuremath${0.0200\pm0.0069}}$
$\newcommand{$\hatcurCCpmra$eccen}{$\ensuremath${-170.503\pm0.028}}$
$\newcommand{$\hatcurCCpmdec$eccen}{$\ensuremath${-64.264\pm0.023}}$
$\newcommand{$\hatcurCCpm$eccen}{$\ensuremath${182.212\pm0.036}}$</div>



<div id="title">

# TOI-3235 b: a transiting giant planet around an M4 dwarf star

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.10008-b31b1b.svg)](https://arxiv.org/abs/2302.10008)<mark>Appeared on: 2023-02-21</mark> - _15 pages, 4 figures. Accepted for publication in APJL_

</div>
<div id="authors">

<mark><mark>Melissa J. Hobson</mark></mark>, et al. -- incl., <mark><mark>Th. Henning</mark></mark>, <mark><mark>Richard P. Schwarz</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present the discovery of TOI-3235 b, a short-period Jupiter orbiting an M-dwarf with a stellar mass close to the critical mass at which stars transition from partially to fully convective. TOI-3235 b was first identified as a candidate from\textit{TESS}photometry, and confirmed with radial velocities from ESPRESSO, and ground-based photometry from HATSouth, MEarth-South, TRAPPIST-South, LCOGT, and ExTrA. We find that the planet has a mass of$\mathrm{$\hatcurPPm$ $\mjup$}$and a radius of$\mathrm{$\hatcurPPr$ $\rjup$}$.  It orbits close to its host star, with an orbital period of$\mathrm{$\hatcurLCP$short d}$, but has an equilibrium temperature of$\mathrm{\approx $\hatcurPPteff$short   K}$, well below the expected threshold for radius inflation of hot Jupiters. The host star has a mass of$\mathrm{$\hatcurISOm$  $\ms$un}$, a radius of$\mathrm{$\hatcurISOr$long  $\rsun$}$, an effective temperature of$\mathrm{$\hatcurISOteff$short   K}$, and a J-band magnitude of$\mathrm{$\hatcurCCtwomass$Jmag}$. Current planet formation models do not predict the existence of gas giants such as TOI-3235 b around such low-mass stars. With a high transmission spectroscopy metric, TOI-3235 b is one of the best-suited giants orbiting M-dwarfs for atmospheric characterization.

</div>

<div id="div_fig1">

<img src="tmp_2302.10008/./TOI3235-hs.png" alt="Fig3.1" width="50%"/><img src="tmp_2302.10008/./TOI3235-lc.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** 
    Ground-based photometry for the the transiting planet system TOI-3235. {\em Left:} Phase-folded unbinned full HATSouth light curve (top), light curve zoomed-in on the transit (middle), and residuals from the best-fit model zoomed-in on the transit (bottom). Solid red lines show the best-fit model. Blue circles show the light curves binned in phase with a bin size of 0.002. {\em Right:} Unbinned follow-up transit light curves corrected for instrumental trends fitted simultaneously with the transit model, which is overplotted (left), and residuals to the fit (right). Dates, filters and instruments are indicated. For ExTrA we indicate the midpoint of the spectral range. The error bars represent the photon
    and background shot noise, plus the readout noise.
\label{fig:toi3235-ground-phot} (*fig:toi3235-ground-phot*)

</div>
<div id="div_fig2">

<img src="tmp_2302.10008/./mass_radius_period_radius_mass_starmass_061_5205.png" alt="Fig1" width="100%"/>

**Figure 1. -** {\em Top}: Mass-radius diagram for M-dwarf planets with masses and radii measured to better than 25\%, as reported in TEPCAT. The markers are colour-coded by host star mass. TOI-3235 b and its analogue TOI-5205 b are plotted with star and hexagon symbols respectively and labelled. Theoretical mass-radius curves from \cite{mordasini:2012} are plotted with dashed and dotted lines. {\em Centre}: Period-radius diagram for the same planets. The markers are scaled by planet mass and colour-coded by equilibrium temperature (black when it could not be computed). TOI-3235 b and TOI-5205 b are labelled. {\em Bottom}: Planet mass vs. stellar mass diagram for the same planets. The markers are colour-coded by host star metallicity. TOI-3235 b and TOI-5205 b are plotted with star and hexagon symbols respectively and labelled. (*fig:rad-mass-period*)

</div>
<div id="div_fig3">

<img src="tmp_2302.10008/./TOI3235-rv.png" alt="Fig4.1" width="50%"/><img src="tmp_2302.10008/./TOI3235-iso-gk-gabs-isofeh-SED.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** {\em Left:}
High-precision RVs from ESPRESSO/VLT phased with respect to the mid-transit time, together with the best-fit model, where the center-of-mass velocity has been subtracted (top); RV $O\!-\!C$ residuals (centre); and bisector spans (bottom). Error bars include the estimated jitter, which is a free parameter in the fitting.
{\em Top Right:} Absolute $G$ magnitude vs. the de-reddened $G - K_{S}$ color from Gaia DR2 and 2MASS (filled blue circle) and $1\sigma$ and $2\sigma$ confidence regions, including estimated systematic errors in the photometry (blue lines), compared to theoretical isochrones (black lines, ages listed in Gyr) and stellar evolution tracks (green dashed lines, mass listed in solar masses) from the MIST models interpolated at the best-estimate value for the host metallicity. The red lines show isochrones at higher and lower metallicities than the best-estimate value, labelled with their metallicity and age in Gyr. {\em Bottom Right:} SED as measured via broadband photometry through the listed filters (top), and $O\!-\!C$ residuals from the best-fit model (bottom). We plot the observed magnitudes without correcting for distance or extinction. Overplotted are 200 model SEDs randomly selected from the MCMC posterior distribution produced through the global analysis (gray lines). Black error bars show the catalog errors for the broad-band photometry measurements; red error bars add an assumed 0.02 mag systematic uncertainty in quadrature to the catalog errors. These latter uncertainties are used in the fit.
\label{fig:toi3235-RV-SED} (*fig:toi3235-RV-SED*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

212  publications in the last 7 days.
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08736.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


# Debugging papers